In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Preprocessing

In [ ]:
!pip install transformers torch pandas

In [ ]:
# Thư viện vncorenlp và wordsegmenter dùng để Word Segmentation cho tiếng Việt

# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

In [ ]:
# tạo bộ word segmentation cho tiếng Việt
from vncorenlp import VnCoreNLP

rdrsegmenter = VnCoreNLP("/kaggle/working/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg")

In [ ]:
replace_list = {
    'ô kêi': 'ok', 'okie': 'ok', 'o kê': 'ok', 'okey': 'ok', 'ôkê': 'ok', 'oki': 'ok', 'oke': 'ok', 'okay': 'ok', 'okê': 'ok',
    'tks': 'cảm ơn', 'thks': 'cảm ơn', 'thanks': 'cảm ơn', 'ths': 'cảm ơn', 'thank': 'cảm ơn',
    'kg': 'không', 'not': 'không', 'k': 'không', 'kh': 'không', 'kô': 'không', 'hok': 'không', 'ko': 'không', 'khong': 'không', 'kp': 'không phải',
    'he he': 'tích cực', 'hehe': 'tích cực', 'hihi': 'tích cực', 'haha': 'tích cực', 'hjhj': 'tích cực', 'thick': 'tích cực',
    'lol': 'tiêu cực', 'cc': 'tiêu cực', 'huhu': 'tiêu cực', 'cute': 'dễ thương',

    'sz': 'cỡ', 'size': 'cỡ',
    'wa': 'quá', 'wá': 'quá', 'qá': 'quá',
    'đx': 'được', 'dk': 'được', 'dc': 'được', 'đk': 'được', 'đc': 'được',
    'vs': 'với', 'j': 'gì', '“': ' ', 'time': 'thời gian', 'm': 'mình', 'mik': 'mình', 'r': 'rồi', 'bjo': 'bao giờ', 'very': 'rất',

    'authentic': 'chuẩn chính hãng', 'aut': 'chuẩn chính hãng', 'auth': 'chuẩn chính hãng', 'date': 'hạn sử dụng', 'hsd': 'hạn sử dụng',
    'store': 'cửa hàng', 'sop': 'cửa hàng', 'shopE': 'cửa hàng', 'shop': 'cửa hàng',
    'sp': 'sản phẩm', 'product': 'sản phẩm', 'hàg': 'hàng',
    'ship': 'giao hàng', 'delivery': 'giao hàng', 'síp': 'giao hàng', 'order': 'đặt hàng',

    'gud': 'tốt', 'wel done': 'tốt', 'good': 'tốt', 'gút': 'tốt', 'tot': 'tốt', 'nice': 'tốt', 'perfect': 'rất tốt',
    'quality': 'chất lượng', 'chất lg': 'chất lượng', 'chat': 'chất', 'excelent': 'hoàn hảo', 'bt': 'bình thường',
    'sad': 'tệ', 'por': 'tệ', 'poor': 'tệ', 'bad': 'tệ',
    'beautiful': 'đẹp tuyệt vời', 'dep': 'đẹp',
    'xau': 'xấu', 'sấu': 'xấu',

    'thik': 'thích', 'iu': 'yêu', 'fake': 'giả mạo',
    'quickly': 'nhanh', 'quick': 'nhanh', 'fast': 'nhanh',
    'fresh': 'tươi', 'delicious': 'ngon',

    'dt': 'điện thoại', 'fb': 'facebook', 'face': 'facebook', 'ks': 'khách sạn', 'nv': 'nhân viên',
    'nt': 'nhắn tin', 'ib': 'nhắn tin', 'tl': 'trả lời', 'trl': 'trả lời', 'rep': 'trả lời',
    'fback': 'feedback', 'fedback': 'feedback',
    'sd': 'sử dụng', 'sài': 'xài',
    'ctrai':'con trai',
'khôg':'không',
'bme':'bố mẹ',
'cta':'chúng ta',
'mih':'mình',
'mqh':'mối quan hệ',
'cgai':'con gái',
'nhữg':'những',
'mng':'mọi người',
'svtn':'sinh viên tình nguyện',
'r':'rồi',
'qtam':'quan tâm',
'thươg':'thương',
'qtâm':'quan tâm',
'chug':'chung',
'trườg':'trường',
'thoy':'thôi',
'đki':'đăng ký',
'atsm':'ảo tưởng sức mạnh',
'ạk':'ạ',
'cv':'công việc',
'vch':'vãi chưởng',
'cùg':'cùng',
'pn':'bạn',
'pjt':'biết',
'thjk':'thích',
'keke':'ce ce',
'ktra':'kiểm tra',
'nek':'nè',
'cgái':'con gái',
'nthe':'như thế',
'chúg':'chúng',
'kái':'cái',
'tìh':'tình',
'phòg':'phòng',
'lòg':'lòng',
'từg':'từng',
'rằg':'rằng',
'sốg':'sống',
'thuj':'thôi',
'thuơng':'thương',
'càg':'càng',
'đky':'đăng ký',
'bằg':'bằng',
'sviên':'sinh viên',
'ák':'á',
'đág':'đáng',
'nvay':'như vậy',
'nhjeu':'nhiều',
'xg':'xuống',
'zồi':'rồi',
'trag':'trang',
'zữ':'dữ',
'atrai':'anh trai',
'kte':'kinh tế',
'độg':'động',
'lmht':'liên minh huyền thoại',
'gắg':'gắng',
'đzai':'đẹp trai',
'thgian':'thời gian',
'plz':'pờ ly',
'đồg':'đồng',
'btrai':'bạn trai',
'nthê':'như thế',
'hìhì':'hì hì',
'vọg':'vọng',
'hihe':'hi he',
'đôg':'đông',
'răg':'răng',
'thườg':'thường',
'tcảm':'tình cảm',
'đứg':'đứng',
'ksao':'không sao',
'dz':'đẹp trai',
'hjxhjx':'hix hix',
'cmày':'chúng mày',
'xuốg':'xuống',
'nkư':'như',
'lquan':'liên quan',
'tiếg':'tiếng',
'hajz':'hai',
'xih':'xinh',
'hìh':'hình',
'thàh':'thành',
'ngke':'nghe',
'dzậy':'dậy',
'teencode':'tin cốt',
'tnào':'thế nào',
'tưởg':'tưởng',
'ctrinh':'chương trình',
'phog':'phong',
'hôg':'không',
'zìa':'gì',
'kũg':'cũng',
'ntnao':'như thế nào',
'trọg':'trọng',
'nthế':'như thế',
'năg':'năng',
'ngđó':'người đó',
'lquen':'làm quen',
'riêg':'riêng',
'ngag':'ngang',
'hêhê':'hê hê',
'bnhiu':'bao nhiêu',
'ngốk':'ngốc',
'kậu':'cậu',
'highland':'hai lừn',
'kqua':'kết quả',
'htrc':'hôm trước',
'địh':'định',
'gđình':'gia đinh',
'giốg':'giống',
'csống':'cuộc sống',
'xug':'xùng',
'zùi':'rồi',
'bnhiêu':'bao nhiêu',
'cbị':'chuẩn bị',
'kòn':'còn',
'buôg':'buông',
'csong':'cuộc sống',
'chàg':'chàng',
'chăg':'chăng',
'ngàh':'ngành',
'llac':'liên lạc',
'nkưng':'nhưng',
'nắg':'nắng',
'tíh':'tính',
'khoảg':'khoảng',
'thík':'thích',
'ngđo':'người đó',
'ngkhác':'người khác',
'thẳg':'thẳng',
'kảm':'cảm',
'dàh':'dành',
'júp':'giúp',
'lặg':'lặng',
'vđê':'vấn đề',
'bbè':'bạn bè',
'bóg':'bóng',
'dky':'đăng ký',
'dòg':'dòng',
'uốg':'uống',
'tyêu':'tình yêu',
'snvv':'sinh nhật vui vẻ',
'đthoại':'điện thoại',
'qhe':'quan hệ',
'cviec':'công việc',
'tượg':'tượng',
'qà':'quà',
'thjc':'thích',
'nhưq':'nhưng',
'cđời':'cuộc đời',
'bthường':'bình thường',
'zà':'già',
'đáh':'đánh',
'xloi':'xin lỗi',
'zám':'dám',
'qtrọng':'quan trọng',
'bìh':'bình',
'lzi':'làm gì',
'qhệ':'quan hệ',
'đhbkhn':'đại học bách khoa hà nội',
'hajzz':'hai',
'kủa':'của',
'lgi':'làm gì',
'nvậy':'như vậy',
'qả':'quả',
'đkiện':'điều kiện',
'nèk':'nè',
'tlai':'tương lai',
'bsĩ':'bác sĩ',
'hkì':'học kỳ',
'đcsvn':'đảng cộng sản việt nam',
'vde':'vấn đề',
'chta':'chúng ta',
'òy':'rồi',

    '^_^': 'tích cực', ':)': 'tích cực', ':(': 'tiêu cực',
    '❤️': 'tích cực', '👍': 'tích cực', '🎉': 'tích cực', '😀': 'tích cực', '😍': 'tích cực', '😂': 'tích cực', '🤗': 'tích cực', '😙': 'tích cực', '🙂': 'tích cực',
    '😔': 'tiêu cực', '😓': 'tiêu cực',
    '⭐': 'star', '*': 'star', '🌟': 'star',
}

In [ ]:
# import re
# import emoji
# import string
# emoji_pattern = re.compile("["
#                 u"\U0001F600-\U0001F64F"  # emoticons
#                 u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                 u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                 u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                 u"\U00002702-\U000027B0"
#                 u"\U000024C2-\U0001F251"
#                 u"\U0001f926-\U0001f937"
#                 u'\U00010000-\U0010ffff'
#                 u"\u200d"
#                 u"\u2640-\u2642"
#                 u"\u2600-\u2B55"
#                 u"\u23cf"
#                 u"\u23e9"
#                 u"\u231a"
#                 u"\u3030"
#                 u"\ufe0f"
#     "]+", flags=re.UNICODE)

# def remove_unnecessary_characters(text):
#     text = text.lower()
#     text = re.sub(r"[^a-zA-Zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ0-9\s]", " ", text)
# #     text = underthesea.word_tokenize(text, format="text")
#     text = re.sub(emoji_pattern, " ", text)
#     text = re.sub(r'([a-z]+?)\1+',r'\1', text)
#     text = re.sub(r"(\w)\s*([" + string.punctuation + "])\s*(\w)", r"\1 \2 \3", text)
#     text = re.sub(r"(\w)\s*([" + string.punctuation + "])", r"\1 \2", text)
#     text = re.sub(r"(\d)([^\d.])", r"\1 \2", text)
#     text = re.sub(r"([^\d.])(\d)", r"\1 \2", text)
#     text = re.sub(f"([{string.punctuation}])([{string.punctuation}])+",r"\1", text)
#     text = text.strip()
#     while text.endswith(tuple(string.punctuation+string.whitespace)):
#         text = text[:-1]
#     while text.startswith(tuple(string.punctuation+string.whitespace)):
#         text = text[1:]
#     text = re.sub(r"\s+", " ", text)
#     return text

In [ ]:
def normalize_acronyms(text):
    words = []
    for word in text.strip().split():
        if word.lower() not in replace_list.keys(): words.append(word)
        else: words.append(replace_list[word.lower()])
    return emoji.demojize(' '.join(words))

In [ ]:
import re
import emoji
import string


def normalize_emoticons(text):
    # Convert all forms of smile emoticons to a standard smile :)
    text = re.sub(r":\)+", ":)", text)  
    text = re.sub(r":\(+", ":(", text)

    return emoji.demojize(text)

def remove_unnecessary_characters(text):
    text = text.lower()
    text = normalize_emoticons(text)
    # Normalize extended repeated characters (e.g., đẹppp -> đẹp)
    text = re.sub(r'([a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]{2,})\1+', r'\1', text)
    text = re.sub(r'([0-9]+)\1+', r'\1', text)  # Normalize extended numbers
    # Remove incorrect spaces around punctuation
#     text = re.sub(r'\s*([,.!?;:()]+)\s*', r'\1 ', text)
    text = text.strip()
    # Reduce multiple spaces to a single space
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[.,]', '', text)

    text = normalize_acronyms(text)
    return text

In [ ]:
import re

# Initialize the VnCoreNLP tokenizer

def segment_vietnamese(text):
    segmented_text = rdrsegmenter.tokenize(text)
    return ' '.join([' '.join(sentence) for sentence in segmented_text])


def preprocess_text(text):
    text= text.lower()
    text = segment_vietnamese(text)
    text = remove_unnecessary_characters(text)
    return text

In [ ]:
import pandas as pd
dfDev= pd.read_csv(r'/kaggle/input/absa-test/Dev.csv')
dfTest= pd.read_csv(r'/kaggle/input/absa-test/Test.csv')
dfTrain= pd.read_csv(r'/kaggle/input/absa-test/Train.csv')

In [ ]:
dfDev

In [ ]:
dfDev= dfDev.copy()
dfTest= dfTest.copy()
dfTrain= dfTrain.copy()

### adding overall aspect

In [ ]:
# Function to map star rating to sentiment
def map_sentiment(stars):
    if stars in [1, 2]:
        return 'Negative'
    elif stars == 3:
        return 'Neutral'
    elif stars in [4, 5]:
        return 'Positive'

In [ ]:
dfDev['label'] = dfDev.apply(lambda row: f"{row['label']}{{OVERALL#{map_sentiment(row['n_star'])}}};", axis=1)

In [ ]:
dfTest['label'] = dfTest.apply(lambda row: f"{row['label']}{{OVERALL#{map_sentiment(row['n_star'])}}};", axis=1)


In [ ]:
dfTrain['label'] = dfTrain.apply(lambda row: f"{row['label']}{{OVERALL#{map_sentiment(row['n_star'])}}};", axis=1)

In [ ]:
dfSA= pd.read_csv(r'/kaggle/input/data38k/ecmdata.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
dfSA['label'] = dfSA['label'].str.lower()
dfSA['label'] = dfSA['label'].replace({'pos': 1, 'neu': 2, 'neg': 0})
dfSA['label'] = label_encoder.fit_transform(dfSA['label'])

In [ ]:
dfSA

In [ ]:
dfSA['comment'] = dfSA['comment'].apply(preprocess_text)

In [ ]:
counts =dfSA['label'].value_counts().sort_index()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
dfSA_train, dfSA_temp = train_test_split(dfSA, test_size=0.2, random_state=42, stratify=dfSA['label'])
dfSA_val, dfSA_test = train_test_split(dfSA_temp, test_size=0.5, random_state=42, stratify=dfSA_temp['label'])

In [ ]:
train_counts = dfSA_train['label'].value_counts().sort_index()
val_counts = dfSA_val['label'].value_counts().sort_index()
test_counts = dfSA_test['label'].value_counts().sort_index()

In [ ]:
counts= dfSA['label'].value_counts().sort_index()

In [ ]:
train_counts,val_counts,test_counts, counts

In [ ]:
from matplotlib import pyplot as plt
labels = ['Negative', 'Positive', 'Neutral']

# Plotting the pie charts side by side
fig, axes = plt.subplots(1, 3, figsize=(24, 8))

# Colors
colors = ['#ff9999','#66b3ff','#99ff99']
explode = (0.1, 0, 0)

# Train set
axes[0].pie(train_counts, explode=explode, colors=colors, autopct='%1.1f%%',
            shadow=True, startangle=90)
axes[0].axis('equal')
axes[0].set_title('Train Set')

# Validation set
axes[1].pie(val_counts, explode=explode, colors=colors, autopct='%1.1f%%',
            shadow=True, startangle=90)
axes[1].axis('equal')
axes[1].set_title('Validation Set')

# Test set
axes[2].pie(test_counts, explode=explode, colors=colors, autopct='%1.1f%%',
            shadow=True, startangle=90)
axes[2].axis('equal')
axes[2].set_title('Test Set')

# Adding a main title
fig.suptitle('Tỷ lệ phần trăm cảm xúc trong dữ liệu')

# Adding a legend to the figure
fig.legend(labels, loc='center right', fontsize='large')

plt.subplots_adjust(right=0.85)
plt.show()


In [ ]:
from matplotlib import pyplot as plt
def plot_simple_pie_chart(sizes, labels, title):
    colors = ['#ff9999','#66b3ff','#99ff99']
    explode = (0.1, 0, 0)  # explode the 1st slice

    plt.figure(figsize=(8, 8))
    plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
            shadow=True, startangle=90)
    plt.axis('equal')
    plt.title(title)
    plt.legend(labels, loc='center right', fontsize='large')
    plt.subplots_adjust(right=0.85)
    plt.show()
labels = ['Negative', 'Positive', 'Neutral']

# Plotting the simple pie charts with the provided counts
plot_simple_pie_chart(counts, labels, 'Label Distribution - Dataset')
# plot_simple_pie_chart(val_counts, labels, 'Label Distribution - Validation Set')
# plot_simple_pie_chart(test_counts, labels, 'Label Distribution - Test Set')

In [ ]:
dfSA_test['label'].value_counts()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


dfSA['sentence_length'] = dfSA['comment'].apply(lambda x: len(x.split()))

plt.figure(figsize=(10, 6))
plt.hist(dfSA['sentence_length'], bins=100, color='pink')
plt.xlabel('Sentence Length (words)')
plt.ylabel('Frequency')
plt.grid(axis='y')

plt.show()

In [ ]:
# Calulate the average sentence length
average_sentence_length = dfSA['sentence_length'].mean()
average_sentence_length

In [ ]:
import pandas as pd

# Assuming dfSA is a pandas dataframe with a 'text' column containing sentences.
# Calculate the sentence length (number of words in each sentence)
dfSA['sentence_length'] = dfSA['comment'].apply(lambda x: len(x.split()))

# Calculate the average sentence length
average_sentence_length = dfSA['sentence_length'].mean()

# Calculate the 5th and 95th percentiles
percentile_5 = dfSA['sentence_length'].quantile(0.05)
percentile_95 = dfSA['sentence_length'].quantile(0.95)

print("Average sentence length:", average_sentence_length)
print("5th percentile sentence length:", percentile_5)
print("95th percentile sentence length:", percentile_95)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import re
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')

In [ ]:
class CSADataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = tokenizer(text, max_length=75, padding='max_length', truncation=True, return_tensors="pt")

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze() #remove all singleton dimensions 1
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }
    
train_sadataset = CSADataset(dfSA_train['comment'].tolist(), dfSA_train['label'].tolist())
train_saloader = DataLoader(train_sadataset, batch_size=24, shuffle=True)
val_sadataset = CSADataset(dfSA_val['comment'].tolist(), dfSA_val['label'].tolist())
val_saloader = DataLoader(val_sadataset, batch_size=24, shuffle=False) 
test_sadataset = CSADataset(dfSA_test['comment'].tolist(), dfSA_test['label'].tolist())
test_saloader = DataLoader(test_sadataset, batch_size=24, shuffle=False)

In [ ]:
for i in train_saloader:
    print(i)
    break

In [ ]:
dfTest['label'][15]

In [ ]:
import torch

In [ ]:
df = pd.read_csv('/kaggle/input/absa-test/Dev.csv')

In [ ]:
# Define the mappings
# aspect_to_index = {'GENERAL': 0, 'SER&ACC': 1, 'SCREEN': 2, 'CAMERA': 3, 'FEATURES': 4, 'BATTERY': 5, 'PERFORMANCE': 6, 'STORAGE': 7, 'DESIGN': 8, 'PRICE': 9, 'OVERALL':10}
sentiment_to_index = {'Positive': 1, 'Negative': 0, 'Neutral': 2, 'None': 3}
aspect_to_index = {'GENERAL': 0, 'SER&ACC': 1, 'SCREEN': 2, 'CAMERA': 3, 'FEATURES': 4, 'BATTERY': 5, 'PERFORMANCE': 6, 'STORAGE': 7, 'DESIGN': 8, 'PRICE': 9}

# Preprocess the labels
def preprocess_labels(label_str):
    labels = re.findall(r'\{(.*?)\}', label_str)
    aspects = torch.zeros(len(aspect_to_index), dtype=torch.long)
    sentiments = torch.full((len(aspect_to_index),), sentiment_to_index['None'], dtype=torch.long)

    for label in labels:
        try:
            if '#' in label:
                aspect, sentiment = label.split('#')
            else:
                aspect = label
                sentiment = 'None'  # Default for 'OTHERS'

            if aspect == 'OTHERS':
                continue  # Skip the "OTHERS" aspect

            idx = aspect_to_index[aspect]
            aspects[idx] = 1
            sentiments[idx] = sentiment_to_index[sentiment]
        except KeyError as e:
            print(f"Error processing label: {label}. Error: {str(e)}")
            continue

    return aspects, sentiments

In [ ]:
preprocess_labels('{FEATURES#Negative};{GENERAL#Positive};{OTHERS};')

In [ ]:
# Apply preprocessing
dfTest['comment'] = dfTest['comment'].apply(preprocess_text)
dfTest['aspects'], dfTest['sentiments'] = zip(*dfTest['label'].apply(preprocess_labels))
# Apply preprocessing
dfTrain['comment'] = dfTrain['comment'].apply(preprocess_text)
dfTrain['aspects'], dfTrain['sentiments'] = zip(*dfTrain['label'].apply(preprocess_labels))
# Apply preprocessing
dfDev['comment'] = dfDev['comment'].apply(preprocess_text)
dfDev['aspects'], dfDev['sentiments'] = zip(*dfDev['label'].apply(preprocess_labels))

In [ ]:
dfTest['sentiments'][250]

In [ ]:
dfTest

In [ ]:
def aggregate_sentiment_counts(df):
    sentiment_counts = torch.zeros((len(aspect_to_index), len(sentiment_to_index)), dtype=torch.long)
    for sentiments in df['sentiments']:
        for aspect_idx, sentiment in enumerate(sentiments):
            sentiment_counts[aspect_idx, sentiment] += 1
    return sentiment_counts

train_sentiment_counts = aggregate_sentiment_counts(dfTrain)
val_sentiment_counts = aggregate_sentiment_counts(dfDev)
test_sentiment_counts = aggregate_sentiment_counts(dfTest)

In [ ]:
def plot_sentiment_distribution(train_counts, val_counts, test_counts):
    aspects = list(aspect_to_index.keys())
    sentiments = ['Negative', 'Positive', 'Neutral']

    fig, axes = plt.subplots(2, 5, figsize=(20, 10), sharey=True)
    axes = axes.flatten()

    for i, aspect in enumerate(aspects):
        ax = axes[i]
        counts_train = train_counts[i].tolist()[:3]
        counts_val = val_counts[i].tolist()[:3]
        counts_test = test_counts[i].tolist()[:3]

        bar_width = 0.25
        bar1 = range(len(counts_train))
        bar2 = [x + bar_width for x in bar1]
        bar3 = [x + bar_width for x in bar2]

        ax.bar(bar1, counts_train, color='#ff9999', width=bar_width, edgecolor='grey', label='Train')
        ax.bar(bar2, counts_val, color='#66b3ff', width=bar_width, edgecolor='grey', label='Dev')
        ax.bar(bar3, counts_test, color='#99ff99', width=bar_width, edgecolor='grey', label='Test')

        # Adding text annotations
        for j in range(len(counts_train)):
            ax.text(bar1[j], counts_train[j] + 1, str(counts_train[j]), ha='center', color='black')
            ax.text(bar2[j], counts_val[j] + 1, str(counts_val[j]), ha='center', color='black')
            ax.text(bar3[j], counts_test[j] + 1, str(counts_test[j]), ha='center', color='black')

        ax.set_title(aspect)
        ax.set_xticks([r + bar_width for r in range(len(counts_train))])
        ax.set_xticklabels(sentiments)

    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right', fontsize='large')
#     plt.suptitle('Sentiment Distribution Across Aspects in Train, Val, and Test Sets')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

In [ ]:
plot_sentiment_distribution(train_sentiment_counts, val_sentiment_counts, test_sentiment_counts)

In [ ]:

# Calculate sentence lengths for each dataset
dfTrain['sentence_length'] = dfTrain['comment'].apply(lambda x: len(x.split()))
dfDev['sentence_length'] = dfDev['comment'].apply(lambda x: len(x.split()))
dfTest['sentence_length'] = dfTest['comment'].apply(lambda x: len(x.split()))

# Plot the combined distribution of sentence lengths for each dataset
plt.figure(figsize=(12, 6))

plt.hist(dfTrain['sentence_length'], bins=50, color='#ff9999', edgecolor='grey', alpha=0.5, label='Train')
plt.hist(dfDev['sentence_length'], bins=50, color='red', edgecolor='grey', alpha=0.5, label='Validation')
plt.hist(dfTest['sentence_length'], bins=50, color='blue', edgecolor='grey', alpha=0.5, label='Test')

plt.xlabel('Sentence Length (words)')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class CommentDataset(Dataset):
    def __init__(self, comments, labels):
        self.comments = comments
        self.labels = labels

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        comment = self.comments[idx]
        aspects, sentiments = preprocess_labels(self.labels[idx])
        inputs = tokenizer(comment, max_length=256, padding='max_length', truncation=True, return_tensors="pt")
        return {
            'input_ids': inputs['input_ids'].squeeze(0), # removes only the first dimension if it is a singleton
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'aspects': aspects,
            'sentiments': sentiments
        }
    
train_dataset = CommentDataset(dfTrain['comment'].tolist(), dfTrain['label'].tolist())
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataset = CommentDataset(dfDev['comment'].tolist(), dfDev['label'].tolist())
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False) 
test_dataset = CommentDataset(dfTest['comment'].tolist(), dfTest['label'].tolist())
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# import torch

# # Assuming encoding contains tensors with singleton dimensions
# encoding = {
#     'input_ids': torch.tensor([[[1, 2, 3]]]),  # Shape: (1, 1, 3)
#     'attention_mask': torch.tensor([[[1, 1, 1]]])  # Shape: (1, 1, 3)
# }
# print(encoding['input_ids'])

# print("Input IDs shape:", encoding['input_ids'].shape) 
# print("Attention Mask shape:", encoding['attention_mask'].shape)

# input_ids = encoding['input_ids'].squeeze()
# attention_mask = encoding['attention_mask'].squeeze()
# print(input_ids)
# print("Input IDs shape:", input_ids.shape)  # Output: torch.Size([3])
# print("Attention Mask shape:", attention_mask.shape)  # Output: torch.Size([3])


In [ ]:
# import torch

# # Assuming inputs contains tensors with a batch dimension of size 1
# inputs = {
#     'input_ids': torch.tensor([[1, 2, 3]]),  # Shape: (1, 3)
#     'attention_mask': torch.tensor([[1, 1, 1]])  # Shape: (1, 3)
# }

# processed_inputs = {
#     'input_ids': inputs['input_ids'].squeeze(0),
#     'attention_mask': inputs['attention_mask'].squeeze(0),
# }

# print("Input IDs shape:", processed_inputs['input_ids'].shape)  # Output: torch.Size([3])
# print("Attention Mask shape:", processed_inputs['attention_mask'].shape)  # Output: torch.Size([3])


In [ ]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# Example sentences
sentences = ["This is a test.", "Another test sentence."]

# Tokenize with batching
encoding = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
print(encoding['input_ids']) #Each number corresponds to a token in the sentence according to the tokenizer’s vocabulary
print(encoding['attention_mask']) # 1s for positions with actual tokens and 0s for positions with padding tokens.
print("Input IDs shape:", encoding['input_ids'].shape)
print("Attention Mask shape:", encoding['attention_mask'].shape)


In [ ]:
for i in test_saloader:
    print(i)
    break

In [ ]:
for i in test_loader:
    print(i)
    break

In [ ]:
import torch.nn as nn
from transformers import AutoModel

In [ ]:
# import torch
# import torch.nn as nn

# # Class counts
# class_counts = [8737, 22554, 5521]

# # Calculate class weights
# class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)
# class_weights = class_weights / class_weights.sum()  # Normalize the weights
# class_weights = class_weights.to(device)  # Move to the same device as your model

# # Define the criterion with the calculated weights
# criterion_csa = nn.CrossEntropyLoss(weight=class_weights)

# # Print the weights for verification
# print(f"Class weights: {class_weights}")


In [ ]:
# # Define the training function as described previously
# def train(model, train_loader,train_saloader, val_loader,val_saloader, optimizer, scheduler, device, num_epochs=20, patience=5):
#     train_losses, val_losses = [], []
#     model.to(device)
#     criterion_aspect = nn.BCEWithLogitsLoss()
#     criterion_sentiment = nn.CrossEntropyLoss()    
#     criterion_csa = nn.CrossEntropyLoss(weight=class_weights)
#     best_val_loss = float('inf')
#     epochs_without_improvement = 0

#     for epoch in range(num_epochs):
#         model.train()
#         total_train_loss = 0
#         for batch in train_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             aspect_labels = batch['aspects'].to(device).float()  # shape: (batch_size, n_aspects)
#             sentiment_labels = batch['sentiments'].to(device).long()

#             optimizer.zero_grad()
#             logits,_ = model(input_ids, attention_mask)
            
#             # Aspect detection loss
#             aspect_logits = logits[:, :, 0]  # Assuming the first column in logits is for aspect detection
# #             print(aspect_logits,'and',aspect_labels)

#             aspect_loss = criterion_aspect(aspect_logits, aspect_labels)
# #             print(aspect_logits,'111:\n', aspect_logits)
            
#             # Sentiment classification loss
#             sentiment_logits = logits.view(-1,4)  # Flatten the logits for CrossEntropyLoss
# #             print(sentiment_logits, 'and1',sentiment_labels.view(-1))

#             sentiment_loss = criterion_sentiment(sentiment_logits, sentiment_labels.view(-1))
# #             print(sentiment_logits,'111:\n', sentiment_labels.view(-1))
    
#             # Total loss
#             absa_loss = aspect_loss + sentiment_loss
#             absa_loss.backward()
#             optimizer.step()
#             scheduler.step()
#             total_train_loss += absa_loss.item()
            
#         for batch in train_saloader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
#             optimizer.zero_grad()

#             _, csa_logits = model(input_ids, attention_mask)
# #             print(csa_logits,'and2',labels)
         
#             csa_loss = criterion_csa(csa_logits, labels)
            
#             csa_loss.backward()
#             optimizer.step()
#             scheduler.step()
#             total_train_loss += csa_loss.item()

#         avg_train_loss = total_train_loss / (len(train_loader) + len(train_saloader))
#         train_losses.append(avg_train_loss)
#         print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train absa Loss: {absa_loss:.4f}, Train csa Loss: {csa_loss:.4f}')

#         model.eval()
#         total_train_val_loss = 0
#         with torch.no_grad():
#             for batch in val_loader:
#                 input_ids = batch['input_ids'].to(device)
#                 attention_mask = batch['attention_mask'].to(device)
#                 aspect_labels = batch['aspects'].to(device).float()  # shape: (batch_size, n_aspects)
#                 sentiment_labels = batch['sentiments'].to(device).long()
                
#                 logits,_ = model(input_ids, attention_mask)

#                 aspect_logits = logits[:, :, 0]  # Assuming the first column in logits is for aspect detection
#                 aspect_loss = criterion_aspect(aspect_logits, aspect_labels)
#     #             print(aspect_logits,'111:\n', aspect_logits)

#                 # Sentiment classification loss
#                 sentiment_logits = logits.view(-1,4)  # Flatten the logits for CrossEntropyLoss

#                 sentiment_loss = criterion_sentiment(sentiment_logits, sentiment_labels.view(-1))
#     #             print(sentiment_logits,'111:\n', sentiment_labels.view(-1))
#                 # Total loss
#                 absa_loss = aspect_loss + sentiment_loss
#                 total_train_val_loss += absa_loss.item()
                
#             for batch in val_saloader:
#                 input_ids = batch['input_ids'].to(device)
#                 attention_mask = batch['attention_mask'].to(device)
#                 labels = batch['labels'].to(device)

#                 _, csa_logits = model(input_ids, attention_mask)

#                 csa_loss = criterion_csa(csa_logits, labels)

#                 total_train_val_loss += csa_loss.item()

#         avg_val_loss = total_train_val_loss / (len(val_loader) + len(val_saloader))
#         val_losses.append(avg_val_loss)
#         print(f'Validation Loss: {avg_val_loss:.4f},Train absa Loss: {absa_loss:.4f}, Train cá Loss: {csa_loss:.4f}')

#         if avg_val_loss < best_val_loss:
#             best_val_loss = avg_val_loss
#             torch.save(model.state_dict(), 'best_model.pth')
#             epochs_without_improvement = 0
#         else:
#             epochs_without_improvement += 1
#             if epochs_without_improvement >= patience:
#                 print(f"Early stopping after {epochs_without_improvement} epochs without improvement.")
#                 break

#     return train_losses, val_losses

In [ ]:
# for batch in val_saloader:
#     print(batch)
#     break
# #     input_ids = batch['input_ids'].to(device)
# #     attention_mask = batch['attention_mask'].to(device)
# #     labels = batch['labels'].to(device)

In [ ]:
# from transformers import AdamW, get_linear_schedule_with_warmup
# optimizer = AdamW(model.parameters(), lr=2e-5)
# total_steps = len(train_loader) * 20  # Assuming 20 epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.1 * total_steps, num_training_steps=total_steps)

In [ ]:
# train_losses, val_losses = train(model, train_loader,train_saloader, val_loader,val_saloader, optimizer, scheduler, device, num_epochs=20, patience=5)

In [ ]:
# aspect_predictions, aspect_true_labels, absasentiment_predictions, absasentiment_true_labels,csasentiment_predictions,csasentiment_true_labels= evaluate(model, test_loader, test_saloader, device, sentiment_to_index)

In [ ]:
# display_metrics_per_aspect_final(aspect_predictions, aspect_true_labels, absasentiment_predictions,absasentiment_true_labels,  aspect_to_index,sentiment_to_index)

In [ ]:
# from sklearn.metrics import classification_report, accuracy_score
# print(classification_report(csasentiment_true_labels, csasentiment_predictions))

In [ ]:
# dftest= pd.read_csv('/kaggle/input/data38k/ecmdata.csv')

In [ ]:
# dftest['comment'][1]

In [ ]:
# comment= 'Pin kém còn lại miễn chê mua 8/3/2019 tình trạng pin còn 88% có ai giống tôi không'

# predicted_aspects, predicted_csa_sentiments = predict(model, tokenizer, device, comment)

# for i, (aspects, csa_sentiment) in enumerate(zip(predicted_aspects, predicted_csa_sentiments)):
#     print(f"Predicted Aspects and Sentiments: {aspects}")
#     print(f"Predicted CSA Sentiment: {csa_sentiment}")

<!-- # END combined absa+csa TEST -->

In [ ]:
# model.to(device)

In [ ]:
# print(device)

In [ ]:
# from transformers import AdamW, get_linear_schedule_with_warmup
# optimizer1 = AdamW(model.parameters(), lr=2e-5)
# total_steps = len(train_loader) * 40  # Assuming 20 epochs
# scheduler1 = get_linear_schedule_with_warmup(optimizer1, num_warmup_steps=0.1 * total_steps, num_training_steps=total_steps)

In [ ]:
# train_losses, val_losses = train(model, train_loader, val_loader, optimizer1, scheduler1, device, num_epochs=40, patience=5)

#### predict

In [ ]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input)
import torch.nn.functional as F
print(F.softmax(input, dim=1))
target = torch.empty(3, dtype=torch.long).random_(5)
print(target)
output = loss(input, target)
print(output)
output.backward()
# # Example of target with class probabilities
# input = torch.randn(3, 5, requires_grad=True)
# print(input)
# target = torch.randn(3, 5).softmax(dim=1)
# print(target)
# output = loss(input, target)
# print(output)
# output.backward()

<!-- ## endtest -->

# ABSA begin

In [ ]:
# aspect_to_index = {'GENERAL': 0, 'SER&ACC': 1, 'SCREEN': 2, 'CAMERA': 3, 'FEATURES': 4, 'BATTERY': 5, 'PERFORMANCE': 6, 'STORAGE': 7, 'DESIGN': 8, 'PRICE': 9}
aspect_to_index = {'GENERAL': 0, 'SER&ACC': 1, 'SCREEN': 2, 'CAMERA': 3, 'FEATURES': 4, 'BATTERY': 5, 'PERFORMANCE': 6, 'STORAGE': 7, 'DESIGN': 8, 'PRICE': 9, 'OVERALL':10}

sentiment_to_index = {'Positive': 1, 'Negative': 0, 'Neutral': 2, 'None': 3}
index_to_sentiment = {v: k for k, v in sentiment_to_index.items()}

index_to_aspect = {v: k for k, v in aspect_to_index.items()}
aspect_names = list(aspect_to_index.keys())

#### 4 last hidden

In [ ]:
# import torch
# import torch.nn as nn
# from transformers import AutoModel

# class NewApproach(nn.Module):
#     def __init__(self, num_aspects=10, num_polarities=4, hidden_dim=3072):
#         super().__init__()
#         self.phobert = AutoModel.from_pretrained("vinai/phobert-base-v2", output_hidden_states=True)
#         self.mlp = nn.Sequential(
#             nn.Linear(self.phobert.config.hidden_size * 4, hidden_dim),
#             nn.ReLU(),
#             nn.Dropout(0.2)
#         )
#         self.aspect_classifiers = nn.ModuleList([
#             nn.Linear(hidden_dim, num_polarities) for _ in range(num_aspects)
#         ])
#         self.num_aspects = num_aspects
#         self.num_polarities = num_polarities

#     def forward(self, input_ids, attention_mask):
#         outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
#         concat_hidden = torch.cat(outputs.hidden_states[-4:], dim=-1)  # Shape: (batch_size, seq_len, hidden_size*4)
#         cls_output = concat_hidden[:, 0, :]  # Shape: (batch_size, hidden_size*4)
#         cls_output = self.mlp(cls_output)
        
#         # Calculate logits for each aspect
#         aspect_logits = [classifier(cls_output) for classifier in self.aspect_classifiers]
        
#         # Concatenate the aspect logits along the last dimension
#         logits = torch.cat(aspect_logits, dim=1)
#         logits = logits.view(-1, self.num_aspects, self.num_polarities)

#         return logits


In [ ]:
import torch.nn as nn
from transformers import AutoModel

class MultibranchABSA(nn.Module):
    def __init__(self, num_aspects=10, num_polarities=4):
        super().__init__()
        self.phobert = AutoModel.from_pretrained("vinai/phobert-base-v2", output_hidden_states=True)
        self.dropout = nn.Dropout(0.2)
        self.classifiers = nn.ModuleList([
            nn.Linear(self.phobert.config.hidden_size * 4, num_polarities) for _ in range(num_aspects)
        ])

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        concat_hidden = torch.cat(outputs.hidden_states[-4:], dim=-1)  # Shape: (batch_size, seq_len, hidden_size*4)
        cls_output = concat_hidden[:, 0, :]  # Shape: (batch_size, hidden_size*4)
        cls_output = self.dropout(cls_output)
        logits = torch.stack([classifier(cls_output) for classifier in self.classifiers], dim=1)
#         logits = [classifier(cls_output) for classifier in self.classifiers]

        return logits

In [ ]:
import torch.nn as nn
from transformers import AutoModel

class MultitaskABSA(nn.Module):
    def __init__(self, num_aspects=10, num_polarities=4):
        super().__init__()
        self.phobert = AutoModel.from_pretrained("vinai/phobert-base-v2", output_hidden_states=True)
        self.dropout = nn.Dropout(0.2)
        # Instead of a classifier for each aspect, use a single classifier that handles all aspects
        self.classifier = nn.Linear(self.phobert.config.hidden_size * 4, num_aspects * num_polarities)

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        concat_hidden = torch.cat(outputs.hidden_states[-4:], dim=-1)
        cls_output = concat_hidden[:, 0, :]  # Use only the [CLS] output
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)
        # Reshape logits to have dimensions (batch_size, num_aspects, num_polarities)
        logits = logits.view(-1, 10, 4)
        return logits

In [ ]:
class MultitaskABSAConcat(nn.Module):
    def __init__(self, num_aspects=11, num_polarities=4):
        super().__init__()
        self.phobert = AutoModel.from_pretrained("vinai/phobert-base-v2", output_hidden_states=True)
        self.dropout = nn.Dropout(0.2)
        self.aspect_classifiers = nn.ModuleList([
            nn.Linear(self.phobert.config.hidden_size * 4, num_polarities) for _ in range(num_aspects)
        ])
        self.num_aspects= num_aspects
        self.num_polarities= num_polarities

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask,  output_hidden_states=True)
        concat_hidden = torch.cat(outputs.hidden_states[-4:], dim=-1)  # Shape: (batch_size, seq_len, hidden_size*4)
        cls_output = concat_hidden[:, 0, :]  # Shape: (batch_size, hidden_size*4)
        cls_output = self.dropout(cls_output)
        
        # Calculate logits for each aspect
        aspect_logits = [classifier(cls_output) for classifier in self.aspect_classifiers]
        
        # Concatenate the aspect logits along the last dimension
        logits = torch.cat(aspect_logits, dim=1)
        logits = logits.view(-1, self.num_aspects, self.num_polarities)

        return logits

In [ ]:
# Create sample input
batch_size = 16
seq_len = 256
input_ids = torch.randint(0, 1000, (batch_size, seq_len))
attention_mask = torch.ones((batch_size, seq_len))

model_multibranch = MultibranchABSA(num_aspects=10, num_polarities=4)
model_concat = MultitaskABSAConcat(num_aspects=10, num_polarities=4)
model_multitask = MultitaskABSA(num_aspects=10, num_polarities=4)
# Generate outputs
outputs_multibranch = model_multibranch(input_ids, attention_mask)
outputs_concat = model_concat(input_ids, attention_mask)
outputs_multitask = model_multitask(input_ids, attention_mask)

# Print outputs
print("MultibranchABSA output shape:", outputs_multibranch.shape)
print("\nMultitaskABSAConcat output shape:", outputs_concat.shape)
print("\nMultitaskABSA output shape:", outputs_multitask.shape)

In [ ]:
# print("MultibranchABSA output shape:", outputs_multibranch[0])
# print("\nMultitaskABSAConcat output shape:", outputs_concat[0])
# print("\nMultitaskABSA output shape:", outputs_multitask[0])

<!-- ### just use origin: adam with weigh-decay=0.01 -->

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
for i in train_loader:
    print(i)
    break

In [ ]:
# Initialize the model, criterion, and optimizer
model = MultitaskABSAConcat(num_aspects=10, num_polarities=4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# Set device to CUDA if available
model.to(device)

# Example of processing one batch
for batch in train_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    sentiments = batch['sentiments'].to(device)
    print('se',sentiments)

    optimizer.zero_grad()
    outputs = model(input_ids, attention_mask)  # This should be a single tensor now
    print('o',outputs)
    loss = 0
    for i in range(outputs.shape[1]):  # Iterate over each aspect
        loss += criterion(outputs[:, i, :], sentiments[:, i])
        print(outputs[:, i, :])
        print(sentiments[:, i])
        print(loss)

    # Print the outputs and loss
        print("Outputs shape:", outputs.shape)
        print("Loss:", loss.item())
        break  # Just process one batch for this e
    break
'''tensor([[-0.1526, -0.0674,  0.2411,  0.1065],
        [-0.0548, -0.0465,  0.3515,  0.1444],
        [-0.1727,  0.1266,  0.3029,  0.1133],
        [ 0.0375,  0.0809,  0.3087,  0.2032],
        [ 0.0060, -0.0743,  0.2716,  0.1565],
        [-0.0580,  0.1228,  0.1853, -0.0122],
        [-0.0251,  0.0960,  0.3207,  0.1585],
        [ 0.0092,  0.0067,  0.1007,  0.1176],
        [ 0.0576, -0.1359,  0.4195,  0.0727],
        [-0.0487,  0.0264,  0.1734,  0.0826],
        [-0.0885,  0.0263,  0.2352,  0.1001],
        [ 0.0332, -0.0066,  0.2499,  0.2246],
        [-0.0051,  0.0358,  0.1585,  0.0326],
        [-0.0489, -0.0470,  0.3628,  0.0262],
        [-0.0216,  0.1041,  0.2702, -0.0063],
        [-0.0724, -0.0241,  0.1483,  0.1142]], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([3, 1, 3, 2, 3, 2, 1, 0, 0, 0, 0, 3, 2, 2, 3, 0], device='cuda:0')'''

In [ ]:
# import torch
# import torch.nn as nn

# # Sample predictions and labels
# predictions = torch.tensor([[-0.1508, -0.0144, -0.1146, -0.0415],
#                             [-0.0674, -0.0657, -0.0633, -0.0135],
#                             [-0.0055,  0.0223, -0.0400, -0.0950],
#                             [ 0.0037,  0.0225, -0.0924, -0.2033],
#                             [-0.0806,  0.0098, -0.2648, -0.1741],
#                             [-0.1511, -0.0042, -0.1899, -0.1380],
#                             [-0.1143, -0.0272, -0.0723, -0.0984],
#                             [-0.1456, -0.0070, -0.1005, -0.0336],
#                             [-0.2205,  0.0230, -0.0437, -0.0593],
#                             [-0.0005,  0.0460, -0.2211, -0.0957],
#                             [-0.1531,  0.1311, -0.0162, -0.0652],
#                             [-0.0349,  0.0098, -0.0914, -0.0318],
#                             [-0.2264,  0.0451,  0.1416, -0.1238],
#                             [-0.1334,  0.0085, -0.0709, -0.0355],
#                             [-0.0522, -0.0064, -0.1147, -0.0847],
#                             [-0.1338, -0.0464,  0.0042, -0.0322]])

# labels = torch.tensor([1, 1, 1, 1, 3, 0, 2, 1, 3, 1, 1, 3, 0, 0, 1, 3])

# # Initialize the loss function
# criterion = nn.CrossEntropyLoss()

# # Calculate the cross-entropy loss
# loss = criterion(predictions, labels)

# print("Cross-entropy loss:", loss.item())


In [ ]:
# # Initialize the model, criterion, and optimizer
# model = MultitaskABSA(num_aspects=num_aspects, num_polarities=num_polarities)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# # Set device to CUDA if available
# model.to(device)

# # Example of processing one batch
# for batch in train_loader:
#     input_ids = batch[0].to(device)
#     attention_mask = batch[1].to(device)
#     sentiments = batch[2].to(device)
#     print(sentiments)

#     optimizer.zero_grad()
#     outputs = model(input_ids, attention_mask)  # This should be a single tensor now
#     print(outputs)
#     loss = 0
#     for i in range(outputs.shape[1]):  # Iterate over each aspect
#         loss += criterion(outputs[:, i, :], sentiments[:, i])
#         print(outputs[:, i, :])
#         print(sentiments[:, i])

#     # Print the outputs and loss
#     print("Outputs shape:", outputs.shape)
#     print("Loss:", loss.item())
#     break  # Just process one batch for this example
# tensor([[-0.1922, -0.0503,  0.0264, -0.0277],
#         [-0.1669, -0.0355, -0.0943, -0.0140],
#         [-0.0265,  0.0483, -0.0404,  0.0403],
#         [-0.2612, -0.0051, -0.3087, -0.0231],
#         [-0.1960,  0.0429, -0.0926,  0.1341],
#         [-0.0686, -0.0588, -0.2685,  0.0557],
#         [-0.1588, -0.0055, -0.0791, -0.0144],
#         [-0.0283, -0.1484, -0.1583,  0.0392],
#         [-0.3763, -0.0678, -0.0498, -0.0280],
#         [-0.1223, -0.0743, -0.0192,  0.1066],
#         [-0.3028,  0.0597, -0.0796, -0.0477],
#         [-0.0961,  0.0364, -0.0780,  0.0935],
#         [-0.0869, -0.1123, -0.1555,  0.0233],
#         [-0.1418,  0.0212, -0.0107, -0.0382],
#         [-0.0082, -0.1902, -0.1779, -0.0022],
#         [-0.2244, -0.0243, -0.0601, -0.0686]],

In [ ]:
# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader, TensorDataset
# from transformers import AutoModel

# # Create a sample dataset
# batch_size = 16
# seq_len = 256
# num_aspects = 10
# num_polarities = 4

# # Random input data and random sentiment labels
# input_ids = torch.randint(0, 1000, (batch_size, seq_len))
# attention_mask = torch.ones((batch_size, seq_len))
# sentiments = torch.randint(0, num_polarities, (batch_size, num_aspects))

# # Create a DataLoader
# dataset = TensorDataset(input_ids, attention_mask, sentiments)
# train_loader = DataLoader(dataset, batch_size=batch_size)

# # Initialize the model, criterion, and optimizer
# model = MultibranchABSA(num_aspects=num_aspects, num_polarities=num_polarities)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# # Set device to CUDA if available
# model.to(device)

# # Example of processing one batch
# for batch in train_loader:
#     input_ids = batch[0].to(device)
#     attention_mask = batch[1].to(device)
#     sentiments = batch[2].to(device)
#     print(sentiments)

#     optimizer.zero_grad()
#     outputs = model(input_ids, attention_mask)  # This should be a single tensor now
#     print(outputs)
#     loss = 0
#     for i in range(outputs.shape[1]):  # Iterate over each aspect
#         loss += criterion(outputs[:, i, :], sentiments[:, i])
#         print(outputs[:, i, :])
#         print(sentiments[:, i])

#     # Print the outputs and loss
#     print("Outputs shape:", outputs.shape)
#     print("Loss:", loss.item())
#     break  # Just process one batch for this example
#     '''
#     [-0.0660, -0.0504, -0.1315,  0.0818],
#         [ 0.0656, -0.0014, -0.1734,  0.0576],
#         [-0.0932, -0.0394, -0.0433,  0.0731],
#         [-0.0787, -0.0532, -0.2123,  0.0245],
#         [-0.0735, -0.0122, -0.1473,  0.0587],
#         [-0.0469,  0.0221, -0.1044, -0.1553],
#         [-0.0902, -0.1343, -0.1494, -0.1250],
#         [-0.0661, -0.0919, -0.0022, -0.2122],
#         [-0.0051,  0.0075, -0.1667, -0.0824],
#         [-0.1292, -0.0723,  0.0188, -0.0652],
#         [-0.1651,  0.0175, -0.1425, -0.0439],
#         [ 0.0003, -0.0883, -0.1191,  0.2049],
#         [-0.1044, -0.0775, -0.1074, -0.1481],
#         [-0.1015, -0.0562, -0.0971,  0.1014],
#         [ 0.0507, -0.0308, -0.0487, -0.0929],
#         [-0.0877, -0.0031, -0.1237, -0.1592]], device='cuda:0',
#        grad_fn=<SliceBackward0>)
# tensor([3, 1, 3, 2, 3, 2, 1, 0, 0, 0, 0, 3, 2, 2, 3, 0], device='cuda:0')'''

In [ ]:
pip install torch_optimizer

In [ ]:
import torch_optimizer as optim
from torch.optim import Adam

# 

In [ ]:
import torch
import torch.nn as nn
import time
import matplotlib.pyplot as plt

def train(model, train_loader, val_loader, device, num_epochs=20, patience=5):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5, weight_decay= 0.01)
    criterion = nn.CrossEntropyLoss()
    model.to(device)
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    def compute_accuracy(outputs, labels):
        _, preds = torch.max(outputs, dim=2)
        corrects = (preds == labels).sum().item()
        total = labels.numel()
        accuracy = corrects / total
        return accuracy

    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        total_loss = 0
        total_corrects = 0
        total_samples = 0

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            sentiments = batch['sentiments'].to(device)  # Assuming shape (batch_size, num_aspects)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)  # This should be a single tensor now        
            loss = 0
            for i in range(outputs.shape[1]):  # Iterate over each aspect
                loss += criterion(outputs[:, i, :], sentiments[:, i])
 
                
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
            total_corrects += compute_accuracy(outputs, sentiments) * sentiments.size(0)
            total_samples += sentiments.size(0)
        
        avg_train_loss = total_loss / len(train_loader)
        avg_train_accuracy = total_corrects / total_samples
        train_losses.append(avg_train_loss)
        train_accuracies.append(avg_train_accuracy)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {avg_train_accuracy:.4f}')

        # Validation phase
        model.eval()
        val_loss = 0
        val_corrects = 0
        val_samples = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                sentiments = batch['sentiments'].to(device)

                outputs = model(input_ids, attention_mask)
                loss = 0
                for i in range(outputs.shape[1]):
                    loss += criterion(outputs[:, i, :], sentiments[:, i])
                val_loss += loss.item()

                val_corrects += compute_accuracy(outputs, sentiments) * sentiments.size(0)
                val_samples += sentiments.size(0)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_accuracy = val_corrects / val_samples
        val_losses.append(avg_val_loss)
        val_accuracies.append(avg_val_accuracy)

        end_time = time.time()
        epoch_time = end_time - start_time

        print(f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {avg_val_accuracy:.4f}')
        print(f'Epoch {epoch+1} took {epoch_time:.2f} seconds.')
        
        # Early stopping logic
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            print(f"No improvement in validation loss for {epochs_without_improvement} epoch(s).")

            if epochs_without_improvement >= patience:
                print(f"Early stopping after {epochs_without_improvement} epochs without improvement.")
                break

    return train_losses, val_losses, train_accuracies, val_accuracies, epoch + 1  # Return the actual number of epochs

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

def evaluate(model, data_loader, device, sentiment_to_index):
    model.eval()
    aspect_predictions, aspect_true_labels = [], []
    full_predictions, full_true_labels = [], []
    combined_aspect_predictions = []
    combined_aspect_true_labels = []
    combined_full_predictions = []
    combined_full_true_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            sentiments = batch['sentiments'].to(device)
#             print(sentiments)

            outputs = model(input_ids, attention_mask)
#             print(outputs)
            logits = outputs[0] if isinstance(outputs, tuple) else outputs  # Make sure to access the logits correctly
            softmax_logits = F.softmax(logits, dim=-1)  # Apply softmax along the last dimension
            # Ensure that the outputs are being indexed correctly
            if logits.dim() == 3:  # Check if logits are three-dimensional
                batch_aspect_predictions = []
                batch_aspect_true_labels = []
                batch_full_predictions = []
                batch_full_true_labels = []
                for i in range(logits.shape[1]):  # Iterate over each aspect
                    preds = torch.argmax(softmax_logits[:, i, :], dim=1)
#                     print('pred',preds)
                    aspect_pred = (preds != sentiment_to_index['None']).long()
#                     print('pred aspect',aspect_pred)
                    aspect_true = (sentiments[:, i] != sentiment_to_index['None']).long()
#                     print('as label',aspect_true)
                    aspect_predictions.append(aspect_pred.cpu().numpy())
                    aspect_true_labels.append(aspect_true.cpu().numpy())
#                     print(aspect_predictions)
#                     print(aspect_true_labels)
                    full_predictions.append(preds.cpu().numpy())
                    full_true_labels.append(sentiments[:, i].cpu().numpy())
#                     print(full_predictions)
#                     print(full_true_labels)
                    batch_aspect_predictions.append(aspect_pred.cpu().numpy())
                    batch_aspect_true_labels.append(aspect_true.cpu().numpy())
                    batch_full_predictions.append(preds.cpu().numpy())
                    batch_full_true_labels.append(sentiments[:, i].cpu().numpy())
#                     print(batch_aspect_predictions,batch_aspect_true_labels,batch_full_predictions,batch_full_true_labels)
                # Combine aspect and full predictions/true labels for the current batch
                for j in range(batch_full_true_labels[0].shape[0]):  # Iterate over the range of batch size
                    combined_aspect_predictions.append([batch_aspect_predictions[k][j] for k in range(len(batch_aspect_predictions))])
                    combined_aspect_true_labels.append([batch_aspect_true_labels[k][j] for k in range(len(batch_aspect_true_labels))])
                    combined_full_predictions.append([batch_full_predictions[k][j] for k in range(len(batch_full_predictions))])
                    combined_full_true_labels.append([batch_full_true_labels[k][j] for k in range(len(batch_full_true_labels))])
#                 print(np.array(combined_aspect_predictions),'\n',np.array(combined_aspect_true_labels),'\n',np.array(combined_full_predictions),'\n',np.array(combined_full_true_labels))
#                 break
            else:
                raise ValueError("Expected three-dimensional output tensor from model")

    # Convert combined lists to numpy arrays
    combined_aspect_predictions = np.array(combined_aspect_predictions)
    combined_aspect_true_labels = np.array(combined_aspect_true_labels)
    combined_full_predictions = np.array(combined_full_predictions)
    combined_full_true_labels = np.array(combined_full_true_labels)

    return combined_aspect_predictions, combined_aspect_true_labels, combined_full_predictions, combined_full_true_labels

In [ ]:
aspect_predictions4, aspect_true_labels4, full_predictions4, full_true_labels4 = evaluate(model4, test_loader, device, sentiment_to_index) #2e5

In [ ]:
def plot_metrics(train_metrics_concat, val_metrics_concat, train_metrics_multibranch, val_metrics_multibranch, actual_epochs_concat, actual_epochs_multibranch):
    max_epochs = max(actual_epochs_concat, actual_epochs_multibranch)
    epochs_range_concat = range(1, actual_epochs_concat + 1)
    epochs_range_multibranch = range(1, actual_epochs_multibranch + 1)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Plot loss
    ax1.plot(epochs_range_concat, train_metrics_concat['loss'], label='Train Loss - MultiTask', color='blue')
    ax1.plot(epochs_range_concat, val_metrics_concat['loss'], label='Val Loss - MultiTask', color='green')
    ax1.plot(epochs_range_multibranch, train_metrics_multibranch['loss'], label='Train Loss - MultiBranch', color='red')
    ax1.plot(epochs_range_multibranch, val_metrics_multibranch['loss'], label='Val Loss - AMultiBranch', color='orange')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss Over Time')
    ax1.legend()
    
    # Plot accuracy
    ax2.plot(epochs_range_concat, train_metrics_concat['accuracy'], label='Train Accuracy - MultiTask', color='blue')
    ax2.plot(epochs_range_concat, val_metrics_concat['accuracy'], label='Val Accuracy - MultiTask', color='green')
    ax2.plot(epochs_range_multibranch, train_metrics_multibranch['accuracy'], label='Train Accuracy - MultiBranch', color='red')
    ax2.plot(epochs_range_multibranch, val_metrics_multibranch['accuracy'], label='Val Accuracy - MultiBranch', color='orange')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Accuracy Over Time')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# model_concat = MultiTaskABSAConcat(n_aspects, n_labels).to(device)
# model_multibranch = MultiTaskABSAMultiBranch(n_aspects, n_labels).to(device)

# # Train and evaluate both models
# train_metrics_concat = {'loss': [], 'accuracy': []}
# val_metrics_concat = {'loss': [], 'accuracy': []}
# train_metrics_multibranch = {'loss': [], 'accuracy': []}
# val_metrics_multibranch = {'loss': [], 'accuracy': []}

# train_losses, val_losses, train_accuracies, val_accuracies, actual_epochs_concat = train(model_concat, dataloader, val_dataloader, device, num_epochs=20, patience=5)
# train_metrics_concat['loss'] = train_losses
# val_metrics_concat['loss'] = val_losses
# train_metrics_concat['accuracy'] = train_accuracies
# val_metrics_concat['accuracy'] = val_accuracies

# train_losses, val_losses, train_accuracies, val_accuracies, actual_epochs_multibranch = train(model_multibranch, dataloader, val_dataloader, device, num_epochs=20, patience=5)
# train_metrics_multibranch['loss'] = train_losses
# val_metrics_multibranch['loss'] = val_losses
# train_metrics_multibranch['accuracy'] = train_accuracies
# val_metrics_multibranch['accuracy'] = val_accuracies

# # Plot metrics
# plot_metrics(train_metrics_concat, val_metrics_concat, train_metrics_multibranch, val_metrics_multibranch, actual_epochs_concat, actual_epochs_multibranch)

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model1 =MultibranchABSA().to(device)
# model2=MultitaskABSA().to(device)
# # model3=NewApproach().to(device)
model4=MultitaskABSAConcat().to(device)

In [ ]:
# Train and evaluate both models
train_metrics_concat = {'loss': [], 'accuracy': []}
val_metrics_concat = {'loss': [], 'accuracy': []}
train_metrics_multibranch = {'loss': [], 'accuracy': []}
val_metrics_multibranch = {'loss': [], 'accuracy': []}

In [ ]:
model4=MultitaskABSAConcat().to(device)

In [ ]:
train_losses, val_losses, train_accuracies, val_accuracies, actual_epochs_concat = train(model4, train_loader, val_loader, device, num_epochs=20, patience=5)
train_metrics_concat['loss'] = train_losses
val_metrics_concat['loss'] = val_losses
train_metrics_concat['accuracy'] = train_accuracies
val_metrics_concat['accuracy'] = val_accuracies
# Plot metrics
# plot_metrics(train_metrics_concat, val_metrics_concat, train_metrics_multibranch, val_metrics_multibranch, actual_epochs_concat, actual_epochs_multibranch)

In [ ]:
# aspect_predictions4, aspect_true_labels4, full_predictions4, full_true_labels4 = evaluate(model4, test_loader, device, sentiment_to_index) #2e5

In [ ]:
train_losses, val_losses, train_accuracies, val_accuracies, actual_epochs_multibranch = train(model1, train_loader, val_loader, device, num_epochs=20, patience=5)

train_metrics_multibranch['loss'] = train_losses
val_metrics_multibranch['loss'] = val_losses
train_metrics_multibranch['accuracy'] = train_accuracies
val_metrics_multibranch['accuracy'] = val_accuracies

In [ ]:
# aspect_predictions1, aspect_true_labels1, full_predictions1, full_true_labels1 = evaluate(model1, test_loader, device, sentiment_to_index) #2e5

In [ ]:
plot_metrics(train_metrics_concat, val_metrics_concat, train_metrics_multibranch, val_metrics_multibranch, actual_epochs_concat, actual_epochs_multibranch)

In [ ]:
aspect_names = list(aspect_to_index.keys())

In [ ]:
aspect_predictions4, aspect_true_labels4, full_predictions4, full_true_labels4 = evaluate(model4, test_loader, device, sentiment_to_index) #1E5

In [ ]:
aspect_predictions1, aspect_true_labels1, full_predictions1, full_true_labels1 = evaluate(model1, test_loader, device, sentiment_to_index) #1E5

In [ ]:
# Example usage:
report = polar_metrics(full_true_labels1, full_predictions1, aspect_names)
print(report) #1e5

In [ ]:
# Example usage:
report = polar_metrics_none(full_true_labels1, full_predictions1, aspect_names)
print(report) #1e5

In [ ]:
# Example usage:
report = aspect_detection_metrics(aspect_true_labels1, aspect_predictions1, aspect_names)
print(report)#sche 3e5

In [ ]:
report = polar_metrics(full_true_labels4, full_predictions4, aspect_names)
print(report)

In [ ]:
report = polar_metrics_none(full_true_labels4, full_predictions4, aspect_names)
print(report)

In [ ]:
report = aspect_detection_metrics(aspect_true_labels4, aspect_predictions4, aspect_names)
print(report)

In [ ]:
train_losses, val_losses, train_accuracies, val_accuracies, actual_epochs_multitask = train(model2, train_loader, val_loader, device, num_epochs=20, patience=5)

In [ ]:
aspect_predictions2, aspect_true_labels2, full_predictions2, full_true_labels2 = evaluate(model2, test_loader, device, sentiment_to_index) #1E5

In [ ]:
plot_losses(train_losses,val_losses)

In [ ]:
# train_losses, val_losses, train_accuracies, val_accuracies, actual_epochs_multibranch = train(model1, train_loader, val_loader, device, num_epochs=20, patience=5)
# aspect_predictions1, aspect_true_labels1, full_predictions1, full_true_labels1 = evaluate(model1, test_loader, device, sentiment_to_index) #2e5

# train_metrics_multibranch['loss'] = train_losses
# val_metrics_multibranch['loss'] = val_losses
# train_metrics_multibranch['accuracy'] = train_accuracies
# val_metrics_multibranch['accuracy'] = val_accuracies

In [ ]:
import time

In [ ]:
# plot_metrics(train_metrics_concat, val_metrics_concat, train_metrics_multibranch, val_metrics_multibranch, actual_epochs_concat, actual_epochs_multibranch)

In [ ]:
# Example usage:
report = polar_metrics_none(full_true_labels2, full_predictions2, aspect_names)
print(report) #1e5

In [ ]:
# Example usage:
report = polar_metrics(full_true_labels2, full_predictions2, aspect_names)
print(report) #1e5

In [ ]:
# Example usage:
report = aspect_detection_metrics(aspect_true_labels2, aspect_predictions2, aspect_names)
print(report)#sche 3e5

In [ ]:
# # Example usage:
# report = aspect_detection_metrics(aspect_true_labels4, aspect_predictions4, aspect_names)
# print(report)#sche 3e5

In [ ]:
def plot_losses(train_losses, val_losses):
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss', color='red')
    plt.plot(val_losses, label='Validation Loss', color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss Over Epochs')
    plt.show()

In [ ]:
aspect_to_index

In [ ]:
aspect_names = list(aspect_to_index.keys())

In [ ]:
for aspect_idx, aspect_name in enumerate(aspect_names):
    print(aspect_idx,aspect_name)

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def polar_metrics_none(full_true_labels, full_predictions, aspect_names):
    aspect_to_index = {
        'Negative': 0, 
        'Positive': 1, 
        'Neutral': 2
    }
    total_macro_precision = 0
    total_macro_recall = 0
    total_macro_f1 = 0
    total_macro_support = 0
    total_acc = 0
    
    num_aspects = len(aspect_names)
    # Initialize report
    report = "Aspect + Polarity Metrics:\n"
    
    # Generate metrics for each aspect
    for aspect_idx, aspect_name in enumerate(aspect_names):
        sentiment_labels = full_true_labels[:, aspect_idx]
        sentiment_predictions = full_predictions[:, aspect_idx]

        # Filter out 'None' sentiment
        valid_indices = sentiment_labels != 3
        sentiment_labels = sentiment_labels[valid_indices]
        sentiment_predictions = sentiment_predictions[valid_indices]
    
        precision, recall, fscore, support = precision_recall_fscore_support(
            sentiment_labels, sentiment_predictions, labels=[0, 1, 2], zero_division=1
        )
        
        macro_avg = precision_recall_fscore_support(
            sentiment_labels, sentiment_predictions, average='macro', zero_division=1
        )
        weighted_avg = precision_recall_fscore_support(
           sentiment_labels, sentiment_predictions, average='weighted', zero_division=1
        )
        accuracy = accuracy_score(sentiment_labels, sentiment_predictions)
        
        total_macro_precision += macro_avg[0]
        total_macro_recall += macro_avg[1]
        total_macro_f1 += macro_avg[2]
        total_macro_support += np.sum(support)
        total_acc += accuracy
        report += f"Metrics for {aspect_name}:\n"
        report += f"{'Polarity':<15} {'precision':<10} {'recall':<10} {'f1-score':<10} {'support':<10}\n"
    
        for polarity_name, i in aspect_to_index.items():
            report += f"{polarity_name:<15} {precision[i]:<10.2f} {recall[i]:<10.2f} {fscore[i]:<10.2f} {support[i]:<10}\n"
    
        report += (
            f"\n{'accuracy':<15} {accuracy:<10.2f} {'':<10} {'':<10} {np.sum(support):<10}\n"
            f"{'macro avg':<15} {macro_avg[0]:<10.2f} {macro_avg[1]:<10.2f} {macro_avg[2]:<10.2f} {np.sum(support):<10}\n"
            f"{'weighted avg':<15} {weighted_avg[0]:<10.2f} {weighted_avg[1]:<10.2f} {weighted_avg[2]:<10.2f} {np.sum(support):<10}\n"
            f"{'-' * 60}\n"
        )
    # Calculate overall macro averages
    overall_macro_precision = total_macro_precision / num_aspects
    overall_macro_recall = total_macro_recall / num_aspects
    overall_macro_f1 = total_macro_f1 / num_aspects
    overall_acc = total_acc / num_aspects
    
    report += "\nOverall Macro Averages:\n"
    report += f"  Precision: {overall_macro_precision:.4f}, Recall: {overall_macro_recall:.4f}, F1-score: {overall_macro_f1:.4f}, Accuracy: {overall_acc:.4f}\n"
    
    return report

# Example usage:
# report = polar_metrics_none(full_true_labels, full_predictions, aspect_names)
# print(report)


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def polar_metrics(full_true_labels, full_predictions, aspect_names):
    aspect_to_index = {
        'Negative': 0, 
        'Positive': 1, 
        'Neutral': 2,
        'None': 3
    }
    total_macro_precision = 0
    total_macro_recall = 0
    total_macro_f1 = 0
    total_macro_support = 0
    total_acc = 0
    
    num_aspects = len(aspect_names)
    # Initialize report
    report = "Aspect + Polarity Metrics:\n"
    
    # Generate metrics for each aspect
    for aspect_idx, aspect_name in enumerate(aspect_names):
        sentiment_labels = full_true_labels[:, aspect_idx]
        sentiment_predictions = full_predictions[:, aspect_idx]
    
        precision, recall, fscore, support = precision_recall_fscore_support(
            sentiment_labels, sentiment_predictions, labels=[0, 1, 2, 3], zero_division=1
        )
        
        macro_avg = precision_recall_fscore_support(
            sentiment_labels, sentiment_predictions, average='macro', zero_division=1
        )
        weighted_avg = precision_recall_fscore_support(
           sentiment_labels, sentiment_predictions, average='weighted', zero_division=1
        )
        accuracy = accuracy_score(sentiment_labels, sentiment_predictions)
        
        total_macro_precision += macro_avg[0]
        total_macro_recall += macro_avg[1]
        total_macro_f1 += macro_avg[2]
        total_macro_support += np.sum(support)
        total_acc += accuracy
        report += f"Metrics for {aspect_name}:\n"
        report += f"{'Polarity':<15} {'precision':<10} {'recall':<10} {'f1-score':<10} {'support':<10}\n"
    
        for polarity_name, i in aspect_to_index.items():
            report += f"{polarity_name:<15} {precision[i]:<10.2f} {recall[i]:<10.2f} {fscore[i]:<10.2f} {support[i]:<10}\n"
    
        report += (
            f"\n{'accuracy':<15} {accuracy:<10.2f} {'':<10} {'':<10} {np.sum(support):<10}\n"
            f"{'macro avg':<15} {macro_avg[0]:<10.2f} {macro_avg[1]:<10.2f} {macro_avg[2]:<10.2f} {np.sum(support):<10}\n"
            f"{'weighted avg':<15} {weighted_avg[0]:<10.2f} {weighted_avg[1]:<10.2f} {weighted_avg[2]:<10.2f} {np.sum(support):<10}\n"
            f"{'-' * 60}\n"
        )
    # Calculate overall macro averages
    overall_macro_precision = total_macro_precision / num_aspects
    overall_macro_recall = total_macro_recall / num_aspects
    overall_macro_f1 = total_macro_f1 / num_aspects
    overall_acc = total_acc / num_aspects
    
    report += "\nOverall Macro Averages:\n"
    report += f"  Precision: {overall_macro_precision:.4f}, Recall: {overall_macro_recall:.4f}, F1-score: {overall_macro_f1:.4f}, Accuracy: {overall_acc:.4f}\n"
    
    return report

# report = polar_metrics(full_true_labels, full_predictions, aspect_names)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

def aspect_detection_metrics(aspect_true_labels, aspect_predictions, aspect_names):
    report = "Aspect Detection Metrics:\n"

    # Initialize accumulators for macro averages
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    total_accuracy = 0
    num_aspects = len(aspect_names)

    # Generate metrics for each aspect
    for aspect_idx, aspect_name in enumerate(aspect_names):
        aspect_true = aspect_true_labels[:, aspect_idx]
        aspect_pred = aspect_predictions[:, aspect_idx]

        aspect_report_dict = classification_report(
            aspect_true, aspect_pred, digits=4, zero_division=1, output_dict=True
        )

        precision = aspect_report_dict["1"]["precision"]
        recall = aspect_report_dict["1"]["recall"]
        f1_score = aspect_report_dict["1"]["f1-score"]
        accuracy = accuracy_score(aspect_true, aspect_pred)

        # Accumulate macro averages
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
        total_accuracy += accuracy

        # Add aspect metrics to the report
        report += f"  {aspect_name} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}, Accuracy: {accuracy:.4f}\n"

    # Calculate overall macro averages
    overall_macro_precision = total_precision / num_aspects
    overall_macro_recall = total_recall / num_aspects
    overall_macro_f1_score = total_f1_score / num_aspects
    overall_macro_accuracy = total_accuracy / num_aspects

    # Add overall macro averages to the report
    report += "\nOverall Macro Averages:\n"
    report += f"  Precision: {overall_macro_precision:.4f}, Recall: {overall_macro_recall:.4f}, F1-score: {overall_macro_f1_score:.4f}, Accuracy: {overall_macro_accuracy:.4f}\n"

    return report

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model1 =MultibranchABSA().to(device)
# model2=MultitaskABSA().to(device)
# model3=NewApproach().to(device)
# model4=MultitaskABSAConcat().to(device)

In [ ]:
# model = NewApproach()
# model.to(device)

# aspect overall

In [ ]:
aspect_names

In [ ]:
aspect_name1= ['GENERAL',
 'SER&ACC',
 'SCREEN',
 'CAMERA',
 'FEATURES',
 'BATTERY',
 'PERFORMANCE',
 'STORAGE',
 'DESIGN',
 'PRICE']

In [ ]:
train_losses, val_losses, train_accuracies, val_accuracies, actual_epochs = train(model4, train_loader, val_loader, device, num_epochs=20, patience=5)
aspect_predictions, aspect_true_labels, full_predictions, full_true_labels = evaluate(model4, test_loader, device, sentiment_to_index) #2e5

In [ ]:
# aspect_predictions, aspect_true_labels, full_predictions, full_true_labels = evaluate(model4, test_loader, device, sentiment_to_index) #1E5

In [ ]:
# Example usage:
report = polar_metrics(full_true_labels, full_predictions, aspect_name1)
print(report) #1e5

In [ ]:
# Example usage:
report = polar_metrics(full_true_labels, full_predictions, aspect_names)
print(report) #1e5

In [ ]:
# Example usage:
report = aspect_detection_metrics(aspect_true_labels, aspect_predictions, aspect_name1)
print(report)

In [ ]:
# Example usage:
report = aspect_detection_metrics(aspect_true_labels, aspect_predictions, aspect_names)
print(report)

In [ ]:
report = polar_metrics_none(full_true_labels, full_predictions, aspect_name1)
print(report)

In [ ]:
report = polar_metrics_none(full_true_labels, full_predictions, aspect_names)
print(report)

In [ ]:
# # Example usage:
# report = aspect_detection_metrics(aspect_true_labels1, aspect_predictions1, aspect_names)
# print(report)

In [ ]:
# aspect_predictions, aspect_true_labels, full_predictions, full_true_labels = evaluate(model1, test_loader, device, sentiment_to_index) #1E5

In [ ]:
# model3.to(device)

In [ ]:
# model4.to(device)

In [ ]:
# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)


In [ ]:
# from transformers import  get_linear_schedule_with_warmup
# optimizer4 = torch.optim.AdamW(model4.parameters(), lr=1e-5, weight_decay=0.01)
# total_steps = len(train_loader) * 20  # Assuming 20 epochs
# scheduler4 = get_linear_schedule_with_warmup(optimizer4, num_warmup_steps=0.1 * total_steps, num_training_steps=total_steps)

In [ ]:
# total_steps

In [ ]:
# from transformers import AdamW, get_linear_schedule_with_warmup
# optimizer1 = AdamW(model1.parameters(), lr=1e-5)
# total_steps = len(train_loader) * 20  # Assuming 20 epochs
# scheduler1 = get_linear_schedule_with_warmup(optimizer1, num_warmup_steps=0.1 * total_steps, num_training_steps=total_steps)

In [ ]:
# optimizer2 = AdamW(model2.parameters(), lr=2e-5)
# scheduler2 = get_linear_schedule_with_warmup(optimizer2, num_warmup_steps=0.1 * total_steps, num_training_steps=total_steps)

In [ ]:
# from transformers import AdamW, get_linear_schedule_with_warmup
# total_steps = len(train_loader) * 20  # Assuming 20 epochs


In [ ]:
# optimizer3 = AdamW(model3.parameters(), lr=1e-5, weight_decay=0.01)
# scheduler3 = get_linear_schedule_with_warmup(optimizer3, num_warmup_steps=0.1 * total_steps, num_training_steps=total_steps)

In [ ]:
# optimizer4 = AdamW(model4.parameters(), lr=2e-5, weight_decay=0.01 )
# scheduler4 = get_linear_schedule_with_warmup(optimizer4, num_warmup_steps=0.1 * total_steps, num_training_steps=total_steps)

#### MultibranchABSA

In [ ]:
# plot_losses(train_losses, val_losses)#sche e5
# model = MultitaskABSAConcat()
# model.to(device)
# model.load_state_dict(torch.load('/kaggle/working/best_model2.pth'))
# aspect_predictions, aspect_true_labels, full_predictions, full_true_labels = evaluate(model, test_loader, device, sentiment_to_index) #2e5

In [ ]:
# aspect_names = list(aspect_to_index.keys())

In [ ]:
# aspect_names

In [ ]:
# # Example usage:
# report = polar_metrics_none(full_true_labels, full_predictions, aspect_names)
# print(report) #1e5

In [ ]:
import torch

def predict(model, tokenizer, device, comments):
    # Handle both single and multiple comments
    if isinstance(comments, str):
        comments = [comments]  # Ensure the input is iterable

    model.eval()
    inputs = tokenizer(comments, max_length=256, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)  # Outputs shape: (batch_size, num_aspects, num_polarities)
#         print(outputs)
        outputs= torch.softmax(outputs, dim=2)
#         print(outputs)
        predictions = torch.argmax(outputs, dim=2)  # Shape: (batch_size, num_aspects)
#         print(predictions)
        predictions = predictions.cpu().numpy()  # Convert to numpy array for easier handling
#         print(predictions)
    # Reverse mappings for aspects and sentiments
    sentiment_to_index = {'Positive': 1, 'Negative': 0, 'Neutral': 2, 'None': 3}
    index_to_sentiment = {v: k for k, v in sentiment_to_index.items()}
    aspect_to_index = {'GENERAL': 0, 'SER&ACC': 1, 'SCREEN': 2, 'CAMERA': 3, 'FEATURES': 4,
                       'BATTERY': 5, 'PERFORMANCE': 6, 'STORAGE': 7, 'DESIGN': 8, 'PRICE': 9}
    index_to_aspect = {v: k for k, v in aspect_to_index.items()}

    predicted_aspects = []
    # Iterate over each comment's predictions
    for comment_predictions in predictions:
        comment_aspects = []
        for aspect_idx, sentiment_idx in enumerate(comment_predictions):
            aspect = index_to_aspect[aspect_idx]
            sentiment = index_to_sentiment[sentiment_idx]
            comment_aspects.append((aspect, sentiment))
        predicted_aspects.append(comment_aspects)

    return predicted_aspects

In [ ]:
dfTest['comment'][39]

In [ ]:
comment='máy đẹp chạy mượt_mà pin xài ổn sạc 3 tiếng đầy pin giá_cả hợp_lý vì mình có mua dung_lượng icloud nên 32gb xài củng tạm !'


{BATTERY#Positive};{PERFORMANCE#Positive};{STORAGE#Neutral};{DESIGN#Positive};{PRICE#Positive};


In [ ]:
comment= preprocess_text(comment)

In [ ]:
predicted_aspects2 = predict(model2, tokenizer, device, comment)

In [ ]:
print("Predicted Aspects and Sentiments:", predicted_aspects2) #1

In [ ]:
    sentiment_to_index = {'Positive': 1, 'Negative': 0, 'Neutral': 2, 'None': 3}
    index_to_sentiment = {v: k for k, v in sentiment_to_index.items()}
    aspect_to_index = {'GENERAL': 0, 'SER&ACC': 1, 'SCREEN': 2, 'CAMERA': 3, 'FEATURES': 4,
                       'BATTERY': 5, 'PERFORMANCE': 6, 'STORAGE': 7, 'DESIGN': 8, 'PRICE': 9}
    index_to_aspect = {v: k for k, v in aspect_to_index.items()}

# NB

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, f1_score,accuracy_score,recall_score,precision_score

# Define mappings
sentiment_to_index = {'Positive': 1, 'Negative': 0, 'Neutral': 2, 'None': 3}
index_to_sentiment = {v: k for k, v in sentiment_to_index.items()}
aspect_to_index = {'GENERAL': 0, 'SER&ACC': 1, 'SCREEN': 2, 'CAMERA': 3, 'FEATURES': 4,
                   'BATTERY': 5, 'PERFORMANCE': 6, 'STORAGE': 7, 'DESIGN': 8, 'PRICE': 9}
index_to_aspect = {v: k for k, v in aspect_to_index.items()}

# Define the order of aspects
aspects = [index_to_aspect[i] for i in range(len(aspect_to_index))]

dfTrainNB = dfTrain.copy()
dfTestNB = dfTest.copy()
# Convert tensor labels to integers
dfTrainNB['sentiments'] = dfTrainNB['sentiments'].apply(lambda lst: [int(x.item()) for x in lst])
dfTestNB['sentiments'] = dfTestNB['sentiments'].apply(lambda lst: [int(x.item()) for x in lst])

# Extract text and sentiment labels
X_train = dfTrainNB['comment']
X_test = dfTestNB['comment']

# Convert sentiment lists to individual columns
y_train_sentiments = pd.DataFrame(dfTrainNB['sentiments'].tolist(), columns=[f'label_{aspect}' for aspect in aspects])
y_test_sentiments = pd.DataFrame(dfTestNB['sentiments'].tolist(), columns=[f'label_{aspect}' for aspect in aspects])

# Train the sentiment classification model for each aspect
sentiment_pipelines= {}

for aspect in aspects:
    sentiment_column = f'label_{aspect}'
    sentiment_pipeline = make_pipeline(TfidfVectorizer(stop_words=None), MultinomialNB())
    sentiment_pipeline.fit(X_train, y_train_sentiments[sentiment_column])
    sentiment_pipelines[aspect] = sentiment_pipeline

# Evaluate sentiment classification
overall_sentiment_precision = []
overall_sentiment_recall = []
overall_sentiment_f1 = []
overall_sentiment_accuracy = []
for aspect in aspects:
    sentiment_column = f'label_{aspect}'
    sentiment_pipeline = sentiment_pipelines[aspect]
    y_pred = sentiment_pipeline.predict(X_test)
    y_true = y_test_sentiments[sentiment_column]
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    print(f"Sentiment Classification Report for {aspect.capitalize()}:")
    print(classification_report(y_test_sentiments[sentiment_column], y_pred, target_names=[index_to_sentiment[i] for i in range(4)], labels=[0, 1, 2, 3],zero_division=0))
    overall_sentiment_precision.append(precision)
    overall_sentiment_recall.append(recall)
    overall_sentiment_f1.append(f1)
    overall_sentiment_accuracy.append(accuracy)   

average_sentiment_precision = sum(overall_sentiment_precision) / len(overall_sentiment_precision)
average_sentiment_recall = sum(overall_sentiment_recall) / len(overall_sentiment_recall)
average_sentiment_f1 = sum(overall_sentiment_f1) / len(overall_sentiment_f1)
average_sentiment_accuracy = sum(overall_sentiment_accuracy) / len(overall_sentiment_accuracy)

print("\nOverall Sentiment Classification Metrics:")
print(f"  Precision: {average_sentiment_precision:.4f}")
print(f"  Recall: {average_sentiment_recall:.4f}")
print(f"  F1-score: {average_sentiment_f1:.4f}")
print(f"  Accuracy: {average_sentiment_accuracy:.4f}")

In [ ]:
y_pred

In [ ]:
pred_sentiments = {}
for aspect in aspects:
    sentiment_column = f'label_{aspect}'
    sentiment_pipeline = sentiment_pipelines[aspect]
    y_pred = sentiment_pipeline.predict(X_test)
    pred_sentiments[sentiment_column] = y_pred

# Convert sentiment predictions to aspect detection predictions
def sentiment_to_aspect_detection(y_true, y_pred):
    true_aspect_detection = [1 if sentiment != 3 else 0 for sentiment in y_true]
    pred_aspect_detection = [1 if sentiment != 3 else 0 for sentiment in y_pred]
    return true_aspect_detection, pred_aspect_detection

# Calculate and print aspect detection metrics for each aspect
print("Aspect Detection Metrics:")
overall_precision = []
overall_recall = []
overall_f1 = []
overall_accuracy = []

for aspect in aspects:
    sentiment_column = f'label_{aspect}'
    true_aspect_detection, pred_aspect_detection = sentiment_to_aspect_detection(y_test_sentiments[sentiment_column], pred_sentiments[sentiment_column])
    
    accuracy = accuracy_score(true_aspect_detection, pred_aspect_detection)
    precision = precision_score(true_aspect_detection, pred_aspect_detection, zero_division=0)
    recall = recall_score(true_aspect_detection, pred_aspect_detection, zero_division=0)
    f1 = f1_score(true_aspect_detection, pred_aspect_detection, zero_division=0)
    
    overall_precision.append(precision)
    overall_recall.append(recall)
    overall_f1.append(f1)
    overall_accuracy.append(accuracy)
    
    print(f"  {aspect} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, Accuracy: {accuracy:.4f}")

# Calculate overall metrics
average_precision = sum(overall_precision) / len(overall_precision)
average_recall = sum(overall_recall) / len(overall_recall)
average_f1 = sum(overall_f1) / len(overall_f1)
average_accuracy = sum(overall_accuracy) / len(overall_accuracy)

print("\nOverall Aspect Detection Metrics:")
print(f"  Precision: {average_precision:.4f}")
print(f"  Recall: {average_recall:.4f}")
print(f"  F1-score: {average_f1:.4f}")
print(f"  Accuracy: {average_accuracy:.4f}")

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV, cross_val_score

# Define mappings
sentiment_to_index = {'Positive': 1, 'Negative': 0, 'Neutral': 2, 'None': 3}
index_to_sentiment = {v: k for k, v in sentiment_to_index.items()}
aspect_to_index = {'GENERAL': 0, 'SER&ACC': 1, 'SCREEN': 2, 'CAMERA': 3, 'FEATURES': 4,
                   'BATTERY': 5, 'PERFORMANCE': 6, 'STORAGE': 7, 'DESIGN': 8, 'PRICE': 9}
index_to_aspect = {v: k for k, v in aspect_to_index.items()}

# Define the order of aspects
aspects = [index_to_aspect[i] for i in range(len(aspect_to_index))]

dfTrainNB = dfTrain.copy()
dfTestNB = dfTest.copy()
# Convert tensor labels to integers
dfTrainNB['sentiments'] = dfTrainNB['sentiments'].apply(lambda lst: [int(x.item()) for x in lst])
dfTestNB['sentiments'] = dfTestNB['sentiments'].apply(lambda lst: [int(x.item()) for x in lst])

# Extract text and sentiment labels
X_train = dfTrainNB['comment']
X_test = dfTestNB['comment']

# Convert sentiment lists to individual columns
y_train_sentiments = pd.DataFrame(dfTrainNB['sentiments'].tolist(), columns=[f'label_{aspect}' for aspect in aspects])
y_test_sentiments = pd.DataFrame(dfTestNB['sentiments'].tolist(), columns=[f'label_{aspect}' for aspect in aspects])

# Train the sentiment classification model for each aspect
sentiment_pipelines = {}
param_grid = {
    'tfidfvectorizer__max_df': [0.75, 0.8, 0.85],
    'tfidfvectorizer__min_df': [3, 5, 10],
    'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)],
    'multinomialnb__alpha': [0.5, 1.0, 1.5]
}

for aspect in aspects:
    sentiment_column = f'label_{aspect}'
    sentiment_pipeline = make_pipeline(
        TfidfVectorizer(stop_words=None),
        MultinomialNB()
    )
    
    grid_search = GridSearchCV(sentiment_pipeline, param_grid, cv=5, scoring='f1_macro')
    grid_search.fit(X_train, y_train_sentiments[sentiment_column])
    
    sentiment_pipelines[aspect] = grid_search.best_estimator_

# Evaluate sentiment classification
overall_sentiment_precision = []
overall_sentiment_recall = []
overall_sentiment_f1 = []
overall_sentiment_accuracy = []

for aspect in aspects:
    sentiment_column = f'label_{aspect}'
    sentiment_pipeline = sentiment_pipelines[aspect]
    y_pred = sentiment_pipeline.predict(X_test)
    y_true = y_test_sentiments[sentiment_column]
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    print(f"Sentiment Classification Report for {aspect.capitalize()}:")
    print(classification_report(y_test_sentiments[sentiment_column], y_pred, target_names=[index_to_sentiment[i] for i in range(4)], labels=[0, 1, 2, 3],zero_division=0))
    overall_sentiment_precision.append(precision)
    overall_sentiment_recall.append(recall)
    overall_sentiment_f1.append(f1)
    overall_sentiment_accuracy.append(accuracy)

average_sentiment_precision = sum(overall_sentiment_precision) / len(overall_sentiment_precision)
average_sentiment_recall = sum(overall_sentiment_recall) / len(overall_sentiment_recall)
average_sentiment_f1 = sum(overall_sentiment_f1) / len(overall_sentiment_f1)
average_sentiment_accuracy = sum(overall_sentiment_accuracy) / len(overall_sentiment_accuracy)

print("\nOverall Sentiment Classification Metrics:")
print(f"  Precision: {average_sentiment_precision:.4f}")
print(f"  Recall: {average_sentiment_recall:.4f}")
print(f"  F1-score: {average_sentiment_f1:.4f}")
print(f"  Accuracy: {average_sentiment_accuracy:.4f}")

pred_sentiments = {}
for aspect in aspects:
    sentiment_column = f'label_{aspect}'
    sentiment_pipeline = sentiment_pipelines[aspect]
    y_pred = sentiment_pipeline.predict(X_test)
    pred_sentiments[sentiment_column] = y_pred

# Convert sentiment predictions to aspect detection predictions
def sentiment_to_aspect_detection(y_true, y_pred):
    true_aspect_detection = [1 if sentiment != 3 else 0 for sentiment in y_true]
    pred_aspect_detection = [1 if sentiment != 3 else 0 for sentiment in y_pred]
    return true_aspect_detection, pred_aspect_detection

# Calculate and print aspect detection metrics for each aspect
print("Aspect Detection Metrics:")
overall_precision = []
overall_recall = []
overall_f1 = []
overall_accuracy = []

for aspect in aspects:
    sentiment_column = f'label_{aspect}'
    true_aspect_detection, pred_aspect_detection = sentiment_to_aspect_detection(y_test_sentiments[sentiment_column], pred_sentiments[sentiment_column])
    
    accuracy = accuracy_score(true_aspect_detection, pred_aspect_detection)
    precision = precision_score(true_aspect_detection, pred_aspect_detection, zero_division=0)
    recall = recall_score(true_aspect_detection, pred_aspect_detection, zero_division=0)
    f1 = f1_score(true_aspect_detection, pred_aspect_detection, zero_division=0)
    
    overall_precision.append(precision)
    overall_recall.append(recall)
    overall_f1.append(f1)
    overall_accuracy.append(accuracy)
    
    print(f"  {aspect} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, Accuracy: {accuracy:.4f}")

# Calculate overall metrics
average_precision = sum(overall_precision) / len(overall_precision)
average_recall = sum(overall_recall) / len(overall_recall)
average_f1 = sum(overall_f1) / len(overall_f1)
average_accuracy = sum(overall_accuracy) / len(overall_accuracy)

print("\nOverall Aspect Detection Metrics:")
print(f"  Precision: {average_precision:.4f}")
print(f"  Recall: {average_recall:.4f}")
print(f"  F1-score: {average_f1:.4f}")
print(f"  Accuracy: {average_accuracy:.4f}")

In [ ]:
print("\nSample Predictions:")
for text in X_test:
    print(f"Text: {text}")
    for aspect in aspects:
        sentiment_pipeline = sentiment_pipelines[aspect]
        pred = sentiment_pipeline.predict([text])[0]
        print(f"  {aspect.capitalize()}: {index_to_sentiment[pred]}")
    break

In [ ]:
def predict_sentiments(comment):
    predictions = {}
    for aspect in aspects:
        sentiment_pipeline = sentiment_pipelines[aspect]
        sentiment_prediction = sentiment_pipeline.predict([comment])[0]
        predictions[aspect] = index_to_sentiment[sentiment_prediction]
    return predictions

# Example usage of the function
comment = "sản phẩm ok nhưng pin hơi yếu"
predicted_sentiments = predict_sentiments(comment)

print("Predicted Sentiments for the given comment:")
for aspect, sentiment in predicted_sentiments.items():
    print(f"{aspect}: {sentiment}")

In [ ]:
# import pandas as pd
# import torch
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.pipeline import make_pipeline
# from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

# # Define mappings
# sentiment_to_index = {'Positive': 1, 'Negative': 0, 'Neutral': 2, 'None': 3}
# index_to_sentiment = {v: k for k, v in sentiment_to_index.items()}
# aspect_to_index = {'GENERAL': 0, 'SER&ACC': 1, 'SCREEN': 2, 'CAMERA': 3, 'FEATURES': 4,
#                    'BATTERY': 5, 'PERFORMANCE': 6, 'STORAGE': 7, 'DESIGN': 8, 'PRICE': 9}
# index_to_aspect = {v: k for k, v in aspect_to_index.items()}

# aspects = [index_to_aspect[i] for i in range(len(aspect_to_index))]




# # Convert tensor labels to integers
# df['sentiments'] = df['sentiments'].apply(lambda lst: [int(x.item()) for x in lst])
# dfTest['sentiments'] = dfTest['sentiments'].apply(lambda lst: [int(x.item()) for x in lst])


# X_train = df['comment']
# X_test = dfTest['comment']

# y_train_sentiments = pd.DataFrame(df['sentiments'].tolist(), columns=[f'label_{aspect}' for aspect in aspects])
# y_test_sentiments = pd.DataFrame(dfTest['sentiments'].tolist(), columns=[f'label_{aspect}' for aspect in aspects])

# # Train 
# sentiment_pipelines = {}
# for aspect in aspects:
#     sentiment_column = f'label_{aspect}'
#     sentiment_pipeline = make_pipeline(TfidfVectorizer(stop_words=None), MultinomialNB())
#     sentiment_pipeline.fit(X_train, y_train_sentiments[sentiment_column])
#     sentiment_pipelines[aspect] = sentiment_pipeline

# # Predict 
# pred_sentiments = {}
# for aspect in aspects:
#     sentiment_column = f'label_{aspect}'
#     sentiment_pipeline = sentiment_pipelines[aspect]
#     y_pred = sentiment_pipeline.predict(X_test)
#     pred_sentiments[sentiment_column] = y_pred

# def sentiment_to_aspect_detection(y_true, y_pred):
#     true_aspect_detection = [1 if sentiment != 3 else 0 for sentiment in y_true]
#     pred_aspect_detection = [1 if sentiment != 3 else 0 for sentiment in y_pred]
#     return true_aspect_detection, pred_aspect_detection



In [ ]:

# print("Aspect Detection Metrics:")
# overall_precision = []
# overall_recall = []
# overall_f1 = []
# overall_accuracy = []

# for aspect in aspects:
#     sentiment_column = f'label_{aspect}'
#     true_aspect_detection, pred_aspect_detection = sentiment_to_aspect_detection(y_test_sentiments[sentiment_column], pred_sentiments[sentiment_column])
    
#     accuracy = accuracy_score(true_aspect_detection, pred_aspect_detection)
#     precision = precision_score(true_aspect_detection, pred_aspect_detection ,average='macro', zero_division=0)
#     recall = recall_score(true_aspect_detection, pred_aspect_detection, average='macro', zero_division=0)
#     f1 = f1_score(true_aspect_detection, pred_aspect_detection, average='macro', zero_division=0)
    
#     overall_precision.append(precision)
#     overall_recall.append(recall)
#     overall_f1.append(f1)
#     overall_accuracy.append(accuracy)
    
#     print(f"  {aspect} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, Accuracy: {accuracy:.4f}")

# average_precision = sum(overall_precision) / len(overall_precision)
# average_recall = sum(overall_recall) / len(overall_recall)
# average_f1 = sum(overall_f1) / len(overall_f1)
# average_accuracy = sum(overall_accuracy) / len(overall_accuracy)

# print("\nOverall Aspect Detection Metrics:")
# print(f"  Precision: {average_precision:.4f}")
# print(f"  Recall: {average_recall:.4f}")
# print(f"  F1-score: {average_f1:.4f}")
# print(f"  Accuracy: {average_accuracy:.4f}")

# print("\nSentiment Classification Metrics:")
# overall_sentiment_precision = []
# overall_sentiment_recall = []
# overall_sentiment_f1 = []
# overall_sentiment_accuracy = []

# for aspect in aspects:
#     sentiment_column = f'label_{aspect}'
#     y_true = y_test_sentiments[sentiment_column]
#     y_pred = pred_sentiments[sentiment_column]
    
#     accuracy = accuracy_score(y_true, y_pred)
#     precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
#     recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
#     f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
#     overall_sentiment_precision.append(precision)
#     overall_sentiment_recall.append(recall)
#     overall_sentiment_f1.append(f1)
#     overall_sentiment_accuracy.append(accuracy)
    
#     print(f"  {aspect} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
#     print(classification_report(y_true, y_pred, target_names=[index_to_sentiment[i] for i in range(4)], labels=[0, 1, 2, 3], zero_division=0))

# average_sentiment_precision = sum(overall_sentiment_precision) / len(overall_sentiment_precision)
# average_sentiment_recall = sum(overall_sentiment_recall) / len(overall_sentiment_recall)
# average_sentiment_f1 = sum(overall_sentiment_f1) / len(overall_sentiment_f1)
# average_sentiment_accuracy = sum(overall_sentiment_accuracy) / len(overall_sentiment_accuracy)

# print("\nOverall Sentiment Classification Metrics:")
# print(f"  Precision: {average_sentiment_precision:.4f}")
# print(f"  Recall: {average_sentiment_recall:.4f}")
# print(f"  F1-score: {average_sentiment_f1:.4f}")
# print(f"  Accuracy: {average_sentiment_accuracy:.4f}")


In [ ]:
# # Sample
# print("\nSample Predictions:")
# for text in X_test:
#     print(f"Text: {text}")
#     for aspect in aspects:
#         sentiment_pipeline = sentiment_pipelines[aspect]
#         pred = sentiment_pipeline.predict([text])[0]
#         print(f"  {aspect.capitalize()}: {index_to_sentiment[pred]}")
#     break

  multi: Precision: 0.7799, Recall: 0.7843, F1-score: 0.7645, Accuracy: 0.8127  
  branch: Precision: 0.7812, Recall: 0.7684, F1-score: 0.7629, Accuracy: 0.8105  
  concat: Precision: 0.7986, Recall: 0.7924, F1-score: 0.7834, Accuracy: 0.8228



## ABSA end

# SA

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained('vinai/phobert-base-v2', output_hidden_states=True)
        self.drop = nn.Dropout(p=0.2)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        hidden_states = outputs.last_hidden_state  # Tuple of hidden states
        cls_output = hidden_states[:, 0]  # Taking the [CLS] token's representation
        output = self.drop(cls_output)
        logits = self.out(output) 

        return logits 

In [ ]:
modelsa_path = r"/kaggle/input/samodel/model4.pth"
sa_model=  SentimentClassifier(n_classes=3).to(device)
sa_model.load_state_dict(torch.load(modelsa_path, map_location=device))

In [ ]:
def predictSA(model, tokenizer, device, comments):
    model.eval()
    inputs = tokenizer(comments, max_length=256, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probabilities = torch.softmax(outputs, dim=1)  # Convert logits to probabilities
        predictions = torch.argmax(probabilities, dim=1)  # Get predicted class
        predictions = predictions.cpu().numpy()  # Convert to numpy array for easier handling

    # Reverse mappings for sentiments
    index_to_sentiment = {0: 'Negative', 1: 'Positive', 2: 'Neutral'}
    predicted_sentiments = [index_to_sentiment[idx] for idx in predictions]

    return predicted_sentiments

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')


In [ ]:
predict = predict(model, tokenizer, device, comments)


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
model = NewapproachABSA1()
model = model.to(device)

# Load the trained model weights
model_path = '/kaggle/input/test-demo/model7.pth'
model.load_state_dict(torch.load(model_path, map_location=device))

In [ ]:
# checkpoint = torch.load('model_checkpoint.pth', map_location=device)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


In [ ]:
model.eval()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

In [ ]:
def predict(model, tokenizer, device, comments):
    model.eval()
    inputs = tokenizer([comments], max_length=256, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)  # Outputs shape: (1, num_aspects, num_polarities)
        predictions = torch.argmax(outputs, dim=2)  # Shape: (1, num_aspects)
        predictions = predictions.cpu().numpy()  # Convert to numpy array for easier handling

    # Corrected sentiment mapping
    sentiment_to_index = {'Positive': 1, 'Negative': 0, 'Neutral': 2, 'None': 3}
    index_to_sentiment = {v: k for k, v in sentiment_to_index.items()}

    aspect_to_index = {'GENERAL': 0, 'SER&ACC': 1, 'SCREEN': 2, 'CAMERA': 3, 'FEATURES': 4,
                       'BATTERY': 5, 'PERFORMANCE': 6, 'STORAGE': 7, 'DESIGN': 8, 'PRICE': 9, 'OTHERS': 10}
    index_to_aspect = {v: k for k, v in aspect_to_index.items()}

    predicted_aspects = []
    # Assuming single comment prediction, extract predictions for each aspect
    for aspect_idx, sentiment_idx in enumerate(predictions[0]):
        aspect = index_to_aspect[aspect_idx]
        sentiment = index_to_sentiment[sentiment_idx]
        predicted_aspects.append((aspect, sentiment))

    return predicted_aspects

In [ ]:
comments ='sản_phẩm tốt chất_lượng có_thể thay cho tai nghe havit i62 , shop gói hàng cẩn_thận chu_đáo trả_lời tin nhắn nhiệt_tình'
comments1= 'chất_lượng sản_phẩm : okla hàng mall thì khỏi_phải_nói r he , thay vào như mới rcm cực mạnh mua về thay cho tai nghe'
comments2='tính_năng nổi_bật : ôm sát tai hạn_chế đau tai chất_lượng sản_phẩm : ổn cũng dễ gắn ra vô lắm êm , mềm , đẹp nha cho 10đ nhưng_mà phải gỡ cẩn_thận chứ không dễ bị gãy mắc cài tai nghe lắm mn'
comments3='chất_lượng sản_phẩm : ok tính_năng nổi_bật : ok shop rep inb nhanh , nhiệt_tình , đồ giao cũng nhanh , hàng chuẩn , cách_âm tốt , giá ok và chất_lượng cũng khá ổn , sẽ còn ủng_hộ shop nhiều , màu xinh quá'
comments= preprocess_text(comments)
comments1= preprocess_text(comments1)
comments2= preprocess_text(comments2)
comments3= preprocess_text(comments3)
# test ='Tivi to đẹp, màn hình rõ fullhd, nhưng loa nghe hơi rè, ship nhanh, đáng đồng tiền, '
test= 'tai nghe vừa tai không bị đau hay chật, bass rõ ràng nghe rất thích, giá ok '
test=preprocess_text(test)

predicted_aspects = predict(model, tokenizer, device, comments)
predicted_aspects1 = predict(model, tokenizer, device, comments1)
predicted_aspects2 = predict(model, tokenizer, device, comments2)
predicted_aspects3 = predict(model, tokenizer, device, comments3)
test = predict(model, tokenizer, device, test)

print("\nPredicted Aspects and Sentiments:\n", predicted_aspects)
print("\nPredicted Aspects and Sentiments:\n", predicted_aspects1)
print("\nPredicted Aspects and Sentiments:\n", predicted_aspects2)
print("\nPredicted Aspects and Sentiments:\n", predicted_aspects3)
print("\nPredicted Aspects and Sentiments:\n", test)

In [ ]:
comments ='sanr phẩm đẹp, chụp ảnh sịn nhưng pin hơi yếu dùng được có 2 tiếng là hết rồi'
comments= preprocess_text(comments)


predicted_aspects = predict(model, tokenizer, device, comments)
print("Predicted Aspects and Sentiments:", predicted_aspects)


In [ ]:
df['label'][5555]

In [ ]:
df['comment'][5555]

'{FEATURES#Negative};{PRICE#Neutral};{GENERAL#Positive};'


# SA test

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import numpy as np

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=75):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          padding='max_length',
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True
        )

        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'sentiments': torch.tensor(label, dtype=torch.long)  # Changed 'labels' to 'sentiments'
        }


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel

class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained('vinai/phobert-base-v2', output_hidden_states=True)
        self.drop = nn.Dropout(p=0.2)
        # Multipling the hidden size by 4 because we concatenate the last four layers
        self.out = nn.Linear(self.bert.config.hidden_size * 4, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        # Concatenate the last four hidden states
        hidden_states = outputs.hidden_states
        concat_hidden = torch.cat(tuple(hidden_states[-i] for i in range(1, 5)), dim=-1)
        # We use the output of the concatenated last layer's [CLS] tokens
        cls_output = concat_hidden[:, 0, :]  # Taking the [CLS] token's representation
        output = self.drop(cls_output)
        # logits = self.classifier(cls_output) # This should be corrected as below
        logits = self.out(output)  # Changed to `self.out`

        return logits  

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from transformers import AutoModel

# class SentimentClassifier1(nn.Module):
#     def __init__(self, n_classes):
#         super(SentimentClassifier1, self).__init__()
#         self.bert = AutoModel.from_pretrained('vinai/phobert-base', output_hidden_states=True)
#         self.drop = nn.Dropout(p=0.5)
#         # Multipling the hidden size by 4 because we concatenate the last four layers
#         self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

#     def forward(self, input_ids, attention_mask):
#         outputs = self.bert(
#           input_ids=input_ids,
#           attention_mask=attention_mask
#         )
#         # Concatenate the last four hidden states
#         hidden_states = outputs.last_hidden_state  # Tuple of hidden states
# #         concat_hidden = torch.cat(tuple(hidden_states[-i] for i in range(1, 5)), dim=-1)
#         # We use the output of the concatenated last layer's [CLS] tokens
#         cls_output = hidden_states[:, 0]  # Taking the [CLS] token's representation
#         output = self.drop(cls_output)
#         # logits = self.classifier(cls_output) # This should be corrected as below
#         logits = self.out(output)  # Changed to `self.out`

#         return logits 

In [ ]:
# train_sadataset = CSADataset(dfSA_train['comment'].tolist(), dfSA_train['label'].tolist())
# train_saloader = DataLoader(train_sadataset, batch_size=32, shuffle=True)
# val_sadataset = CSADataset(dfSA_val['comment'].tolist(), dfSA_val['label'].tolist())
# val_saloader = DataLoader(val_sadataset, batch_size=32, shuffle=False) 
# test_sadataset = CSADataset(dfSA_test['comment'].tolist(), dfSA_test['label'].tolist())
# test_saloader = DataLoader(test_sadataset, batch_size=32, shuffle=False)

In [ ]:
dfSA_test['label'].value_counts()

In [ ]:
for i in train_saloader:
    print(i)
    break

In [ ]:
def train(model, train_loader, val_loader, optimizer, scheduler, device, num_epochs=20, patience=5):
    criterion = torch.nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    train_losses=[]
    val_losses=[]
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)   #attention labels and sentiments different
            #print(labels,'\n')
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            #print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
        scheduler.step()
        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}')

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        print(f'Validation Loss: {avg_val_loss:.4f}')

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            epochs_without_improvement = 0
        
        else:
            epochs_without_improvement += 1
            print(f'no improvement after {epochs_without_improvement} epochs')
            if epochs_without_improvement >= patience:
                print("Early stopping...")
                break
    return  train_losses, val_losses

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
model = SentimentClassifier(n_classes=3).to(device)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
total_steps = len(train_loader) * 20
# Assuming 20 epochs

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.1 * total_steps, num_training_steps=total_steps)

In [ ]:
train_losses, val_losses = train(model, train_saloader,val_saloader, optimizer, scheduler, device, num_epochs=0, patience=5)

In [ ]:
def plot_losses(train_losses, val_losses):
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss', color='red')
    plt.plot(val_losses, label='Validation Loss', color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss Over Epochs')
    plt.show()

In [ ]:
plot_losses(train_losses, val_losses) #0.2

In [ ]:
# plot_losses(train_losses, val_losses)  #0.3

In [ ]:
import torch
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score

def evaluate(model, data_loader, device):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            # Ensure this is the correct key for your labels. Use 'labels' or 'sentiments' based on your dataset setup
#             labels = batch.get('sentiments', batch.get('labels')).to(device)
            labels = batch.get('labels')  # Assuming 'labels' is the consistent key used
            if labels is None:
                raise ValueError("Label key 'labels' not found in the batch")
            labels = labels.to(device)
            
            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)  # Convert logits to probabilities

            predictions = torch.argmax(probabilities, dim=1)  # Assuming the output is [batch_size, num_classes]

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate overall metrics
    precision, recall, f1_score, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro') #weighted, macro, micro
    accuracy = accuracy_score(all_labels, all_predictions)

    # Return predictions and true labels, along with calculated metrics
    return all_predictions, all_labels, {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }

In [ ]:
for i in test_saloader:
    print(i)
    break

In [ ]:
sentiment_to_index = {'Negative': 0, 'Positive': 1, 'Neutral': 2}  

predictions, true_labels, metrics = evaluate(model, test_saloader, device)
# print("Predictions:", predictions)
# print("True Labels:", true_labels)
print("Metrics:", metrics)

In [ ]:
#4 last
display_metrics(true_labels, predictions)

In [ ]:
def display_metrics(full_true_labels, full_predictions):
    print("Classification Report:")
    print(classification_report(full_true_labels, full_predictions, target_names=['Negative', 'Positive', 'Neutral'], zero_division=0))

In [ ]:
def predict(model, tokenizer, device, comments):
    model.eval()
    inputs = tokenizer(comments, max_length=256, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probabilities = torch.softmax(outputs, dim=1)  # Convert logits to probabilities
        predictions = torch.argmax(probabilities, dim=1)  # Get predicted class
        predictions = predictions.cpu().numpy()  # Convert to numpy array for easier handling

    # Reverse mappings for sentiments
    index_to_sentiment = {0: 'Negative', 1: 'Positive', 2: 'Neutral'}
    predicted_sentiments = [index_to_sentiment[idx] for idx in predictions]

    return predicted_sentiments


In [ ]:
# df['comment'][9981]

In [ ]:
# df['label'][9981]

In [ ]:
comments='máy đang xài bị đơ màng hình bấm không được khoảng 5p ms trở_lại bình_thường mua gần được 1 tháng'

In [ ]:
predict(model, tokenizer, device, comments)

In [ ]:
predict(model4, tokenizer, device, comments)

In [ ]:
predict(model4, tokenizer, device, comments)

In [ ]:
predict(model4, tokenizer, device, comments)

In [ ]:
# pip install torchviz


In [ ]:
# import torch
# import torch.nn as nn
# from transformers import AutoModel
# from torchviz import make_dot

# class MultitaskABSAConcat(nn.Module):
#     def __init__(self, num_aspects=10, num_polarities=4):
#         super().__init__()
#         self.phobert = AutoModel.from_pretrained("vinai/phobert-base-v2", output_hidden_states=True)
#         self.dropout = nn.Dropout(0.5)
#         self.aspect_classifiers = nn.ModuleList([
#             nn.Linear(self.phobert.config.hidden_size * 4, num_polarities) for _ in range(num_aspects)
#         ])
#         self.num_aspects = num_aspects
#         self.num_polarities = num_polarities

#     def forward(self, input_ids, attention_mask):
#         outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
#         concat_hidden = torch.cat(outputs.hidden_states[-4:], dim=-1)  # Shape: (batch_size, seq_len, hidden_size*4)
#         cls_output = concat_hidden[:, 0, :]  # Shape: (batch_size, hidden_size*4)
#         cls_output = self.dropout(cls_output)
        
#         # Calculate logits for each aspect
#         aspect_logits = [classifier(cls_output) for classifier in self.aspect_classifiers]
        
#         # Concatenate the aspect logits along the last dimension
#         logits = torch.cat(aspect_logits, dim=1)
#         logits = logits.view(-1, self.num_aspects, self.num_polarities)

#         return logits

# # Tạo một ví dụ đầu vào
# input_ids = torch.randint(0, 100, (1, 128))  # batch_size=1, seq_len=128
# attention_mask = torch.ones_like(input_ids)

# # Khởi tạo mô hình và tạo đồ thị
# model = MultitaskABSAConcat(num_aspects=10, num_polarities=4)
# logits = model(input_ids, attention_mask)

# # Trực quan hóa mô hình
# dot = make_dot(logits, params=dict(model.named_parameters()))
# dot.format = 'png'
# dot.render('model_architecture')


# NB

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Calculate TF-IDF
tf_idf = TfidfVectorizer(smooth_idf=False, min_df=3)
X_train_tfidf = tf_idf.fit_transform(dfSA_train['comment'])
y_train= dfSA_train['label']
X_val_tfidf= tf_idf.transform(dfSA_test['comment'])
y_val=dfSA_test['label']

In [ ]:
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, roc_auc_score

def get_auc_CV(model, X, y, cv=5, scoring='roc_auc_ovo'):
    roc_auc_scorer = make_scorer(roc_auc_score, multi_class='ovo', needs_proba=True)
    scores = cross_val_score(model, X, y, cv=cv, scoring=roc_auc_scorer)
    return scores.mean()

alpha_range = np.arange(0.01, 10, 0.1)

res = pd.Series(
    [get_auc_CV(MultinomialNB(alpha=i), X_train_tfidf, y_train) for i in alpha_range],
    index=alpha_range
)

# Find the best alpha
best_alpha = np.round(res.idxmax(), 2)
print('Best alpha: ', best_alpha)

# Plot AUC vs. Alpha
plt.figure(figsize=(10, 5))
plt.plot(res)
plt.title('AUC vs. Alpha')
plt.xlabel('Alpha')
plt.ylabel('AUC')
plt.grid(True)
plt.show()

In [ ]:
nb = MultinomialNB(alpha = 0.41,force_alpha = True, fit_prior = False)
nb.fit(X_train_tfidf,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score, recall_score

def eval(model, X_train, y_train, X_test, y_test):
    # Predict on training and test data
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Evaluate the performance on the training set
    print("Training Set Evaluation:")
    print("Confusion Matrix:")
    print(confusion_matrix(y_train, y_pred_train))
    print("Classification Report:")
    print(classification_report(y_train, y_pred_train))

    # Evaluate the performance on the test set
    print("Test Set Evaluation:")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred_test))
    print("Classification Report:")
    print(classification_report(y_test, y_pred_test))
    return y_test, y_pred_test

In [ ]:
print("NB MODEL\n")
y_test, y_pred_test=eval(nb, X_train_tfidf, y_train, X_val_tfidf, y_val)

In [ ]:
def display_metrics(full_true_labels, full_predictions):
    print("Classification Report:")
    print(classification_report(full_true_labels, full_predictions, target_names=['Negative', 'Positive', 'Neutral'], zero_division=0))

In [ ]:
display_metrics(y_test, y_pred_test)


In [ ]:
def predict(model, tokenizer, device, comments):
    model.eval()
    inputs = tokenizer(comments, max_length=256, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probabilities = torch.softmax(outputs, dim=1)  # Convert logits to probabilities
        predictions = torch.argmax(probabilities, dim=1)  # Get predicted class
        predictions = predictions.cpu().numpy()  # Convert to numpy array for easier handling

    # Reverse mappings for sentiments
    index_to_sentiment = {0: 'Negative', 1: 'Positive', 2: 'Neutral'}
    predicted_sentiments = [index_to_sentiment[idx] for idx in predictions]

    return predicted_sentiments

In [ ]:
predict(model, tokenizer, device, comments)
